In [1]:
import os
# Increase the number of threads to 4
os.environ['OMP_NUM_THREADS'] = '4'
# Ignore Hugginfceface's tokenizers warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# import torch
# import torch.nn as nn
# import torch.optim as optima
# from torch.utils.data import DataLoader, Dataset

In [3]:
# class RandomDataset(Dataset):
#     def __init__(self, size):
#         self.data = torch.randn(size, 10)
#     def __len__(self):
#         return len(self.data)
#     def __getitem__(self, index):
#         return self.data[index]

# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(10, 5)
#         self.fc2 = nn.Linear(5, 1)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = nn.functional.relu(x)
#         x = self.fc2(x)
#         return x

In [4]:
# torch.randn(1000, 10)

In [5]:
# dataset = RandomDataset(1000)

In [6]:
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
# for i in dataloader:
#     print(i)


In [8]:
import os
import torch
from datasets import load_dataset

import torch.distributed as dist 
import torch.distributed
import torch.distributed
import torch.distributed
import torch.multiprocessing as mp 
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler
from torch.distributed.fsdp import (FullyShardedDataParallel as FSDP, StateDictType)
import torch.cuda.nccl as nccl 
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import DataLoader 
import time
import tqdm
import bs4
import argparse

In [9]:
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B"
DATASET_NAME = "chwenjun225/top_5_insurance_brands_june_news_and_twitter_only"

In [10]:
SYSTEM_PROMPT = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>


    Environment: ipython
    Tools: brave_search, wolfram_alpha

    # Tool Instructions
    - Always execute python code in messages that you share.
    - When looking for real time information use relevant functions if available else fallback to brave_search


    You have access to the following functions:

    Use the function 'extract_data_from_html_page_source' to: Extract information including (headline, url, opening_text,...) from HTML page source
    {
        "name": "extract_data_from_html_page_source",
        "description": "Extract information including (headline, url, opening_text, hit_sentence, source, influencer, country, subregion, language, desktop_reach, mobile_reach, sentiment, key_phrases) from HTML page source",
        "parameters": {
            "html_page_source": {
                "param_type": "string",
                "description": "The HTML page source contain information to extract", 
                "required": true
            }
        }
    }


    If a you choose to call a function ONLY reply in the following format:
    <{start_tag}={function_name}>{parameters}{end_tag}
    where

    start_tag => `<function`
    parameters => a JSON dict with the function argument name as key and function argument value as value. 
    end_tag => `</function>`

    Here is an example,
    <function=example_function_name>{"example_name": "example_value"}</function>

    Reminder:
    - Function calls MUST follow the specified format
    - Required parameters MUST be specified
    - Only call one function at a time
    - Put the entire function call reply on one line"
    - Always add your sources when using search results to answer the user query

    You are a helpful Assistant.<|eot_id|>
    """

USER_PROMPT = """
    <|start_header_id|>user<|end_header_id|>

    Extract information including (URL, HTML source, Headline, Opening Text, Hit Sentence, Source, Influencer, Country, Language) from this HTML page source:
    {} 

    <|eot_id|>
    """

ASSISTANT_PROMPT = """
    <|start_header_id|>assistant<|end_header_id|>
    {}
    <eot_id>
    """

MY_TEST_HTML_PAGE_SOURCE = """
    <!DOCTYPE html>
    <html id="XF" lang="vi-VN" dir="LTR" data-app="public" data-template="thread_view" data-container-key="node-36" data-content-key="thread-5281" data-logged-in="false" data-cookie-prefix="xf_" data-csrf="1723178075,3a6deb8fb7fcf51c78bb639afad7a146" data-nav-section data-page-section="forums" class="has-no-js template-thread_view page-section-forums ">
    <head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover">
    <title>Làm an toàn thông tin thì học gì? | WhiteHat.vn</title>
    <link rel="manifest" href="/webmanifest.php">
    <meta name="theme-color" content="#202125" />
    <meta name="apple-mobile-web-app-title" content="WhiteHat.vn">
    <link rel="apple-touch-icon" href="/data/assets/logo/wh-logo.png" />
    <meta name="description" content="An toàn thông tin là làm gì, nên học những gì? Đó là những câu hỏi mà nhiều bạn dự định theo lĩnh vực an toàn thông tin còn băn khoăn. Do đó trong bài này..." />
    <meta property="og:description" content="An toàn thông tin là làm gì, nên học những gì? Đó là những câu hỏi mà nhiều bạn dự định theo lĩnh vực an toàn thông tin còn băn khoăn. Do đó trong bài này mình xin chia sẻ bài viết &quot;Làm an toàn thông tin thì học gì?&quot; của tác giả Dương Ngọc Thái - một kỹ sư bảo mật ở Google, đồng tác giả các kĩ..." />
    <meta property="twitter:description" content="An toàn thông tin là làm gì, nên học những gì? Đó là những câu hỏi mà nhiều bạn dự định theo lĩnh vực an toàn thông tin còn băn khoăn. Do đó trong bài này mình xin chia sẻ bài viết &quot;Làm an toàn..." />
    <meta property="og:url" content="https://whitehat.vn/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/" />
    <link rel="canonical" href="https://whitehat.vn/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/" />
    <link rel="next" href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2" />
    <meta property="og:site_name" content="WhiteHat.vn" />
    <meta property="og:type" content="website" />
    <meta property="og:title" content="Làm an toàn thông tin thì học gì?" />
    <meta property="twitter:title" content="Làm an toàn thông tin thì học gì?" />
    <meta property="og:image" content="https://whitehat.vn/data/assets/logo/wh-logo.png" />
    <meta property="twitter:image" content="https://whitehat.vn/data/assets/logo/wh-logo.png" />
    <meta property="twitter:card" content="summary" />
    <link rel="preload" href="/styles/fonts/fa/fa-regular-400.woff2?_v=5.15.1" as="font" type="font/woff2" crossorigin="anonymous" />
    <link rel="preload" href="/styles/fonts/fa/fa-solid-900.woff2?_v=5.15.1" as="font" type="font/woff2" crossorigin="anonymous" />
    <link rel="preload" href="/styles/fonts/fa/fa-brands-400.woff2?_v=5.15.1" as="font" type="font/woff2" crossorigin="anonymous" />
    <link rel="stylesheet" href="/css.php?css=public%3Anormalize.css%2Cpublic%3Afa.css%2Cpublic%3Acore.less%2Cpublic%3Aapp.less&amp;s=5&amp;l=1&amp;d=1710560919&amp;k=e11ff421fe72834f8ce7b3b80e2d946897751dfe" />
    <link rel="stylesheet" href="/css.php?css=public%3ACMTV_Math.less%2Cpublic%3Abb_code.less%2Cpublic%3AdcMoreThread.less%2Cpublic%3Afooter_custom.less%2Cpublic%3Amessage.less%2Cpublic%3Anulumia_breadcrumb_essentials.less%2Cpublic%3Ashare_controls.less%2Cpublic%3Axc_user_activity.less%2Cpublic%3Aextra.less&amp;s=5&amp;l=1&amp;d=1710560919&amp;k=469ff5fd596ab0df3d397f29b984c9638919bb37" />
    <style>

        .sticky {
            position: fixed;
            top: 20px;
        }

    </style>
    <style>

    .Banner-2ben {
        position: relative;
        display:none;
    }
    .banner_left {
        position: absolute;
        z-index: 9999;
        top: -10px;
        left: -145px;
    }
    .banner_right {
        position: absolute;
        z-index: 9999;
        top: -10px;
        right: -8px;
    }
    .banner_left img, .banner_right img {
            position: fixed;
    }
    @media (max-width: 1360px) {
        .Banner-2ben {
            display:none;
        }
    }

    </style>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/katex@0.12.0/dist/katex.min.css" crossorigin="anonymous">
    <script defer src="https://cdn.jsdelivr.net/npm/katex@0.12.0/dist/katex.min.js" crossorigin="anonymous" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script defer src="https://cdn.jsdelivr.net/npm/katex@0.12.0/dist/contrib/auto-render.min.js" crossorigin="anonymous" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/xf/preamble.min.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-CEM88570J3" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script type="ffcfcb791e2c54b60847998b-text/javascript">
            window.dataLayer = window.dataLayer || [];
            function gtag(){dataLayer.push(arguments);}
            gtag('js', new Date());
            gtag('config', 'G-CEM88570J3', {
                // 
                
                
                    'anonymize_ip': true,
                
            });
        </script>
    </head>
    <body data-template="thread_view">
    <div class="p-pageWrapper" id="top">
    <header class="p-header" id="header">
    <div class="p-header-inner">
    <div class="p-nav-opposite">
    <div class="p-navgroup p-account p-navgroup--guest">
    <a href="/threads/1770/" class="p-navgroup-link p-navgroup-link--textual" data-follow-redirects="on">
    <span class="p-navgroup-linkText">Giới thiệu</span>
    </a>
    <a href="/misc/contact" class="p-navgroup-link p-navgroup-link--textual" data-xf-click="overlay" data-follow-redirects="on">
    <span class="p-navgroup-linkText">Liên hệ</span>
    </a>
    <a href="/login/" class="p-navgroup-link p-navgroup-link--textual p-navgroup-link--logIn" data-overlay-config="{&quot;className&quot;:&quot;custom-overlay&quot;}" data-follow-redirects="on">
    <span class="p-navgroup-linkText">Đăng nhập</span>
    </a>
    <a href="/register/" class="p-navgroup-link p-navgroup-link--textual p-navgroup-link--register" data-overlay-config="{&quot;className&quot;:&quot;custom-overlay&quot;}" data-follow-redirects="on">
    <span class="p-navgroup-linkText">Đăng ký</span>
    </a>
    </div>
    </div>
    <div class="p-header-content">
    <div class="p-header-logo p-header-logo--image">
    <a href="https://whitehat.vn">
    <span>CỘNG ĐỒNG AN NINH MẠNG VIỆT NAM</span>
    </a>
    </div>
    </div>
    </div>
    </header>
    <nav class="p-nav">
    <div class="p-nav-inner">
    <button type="button" class="button--plain p-nav-menuTrigger button" data-xf-click="off-canvas" data-menu=".js-headerOffCanvasMenu" tabindex="0" aria-label="Trình đơn"><span class="button-text">
    <i aria-hidden="true"></i>
    </span></button>
    <div class>
    <a href="https://whitehat.vn">
    <img src="/styles/whitehat-2021/logo.svg" srcset alt="WhiteHat.vn" width height />
    </a>
    </div>
    <div class="p-nav-scroller hScroller" data-xf-init="h-scroller" data-auto-scroll=".p-navEl.is-selected">
    <div class="hScroller-scroll">
    <ul class="p-nav-list js-offCanvasNavSource">
    <li>
    <div class="p-navEl " data-has-children="true">
    <a href="/tin-tuc-an-ninh-mang" class="p-navEl-link p-navEl-link--splitMenu " data-nav-id="tin_tuc">Tin Tức</a>
    <a data-xf-key="1" data-xf-click="menu" data-menu-pos-ref="< .p-navEl" class="p-navEl-splitTrigger" role="button" tabindex="0" aria-label="Toggle expanded" aria-expanded="false" aria-haspopup="true"></a>
    <div class="menu menu--structural" data-menu="menu" aria-hidden="true">
    <div class="menu-content">
    <a href="/tin-tuc-an-ninh-mang" class="menu-linkRow u-indentDepth0 js-offCanvasCopy " data-nav-id="anninhwh">Tin tức An ninh mạng</a>
    <a href="/ban-tin-whitehat" class="menu-linkRow u-indentDepth0 js-offCanvasCopy " data-nav-id="tinwh">Bản tin WhiteHat</a>
    </div>
    </div>
    </div>
    </li>
    <li>
    <div class="p-navEl ">
    <a href="/members/" class="p-navEl-link " data-xf-key="2" data-nav-id="members">Thành viên</a>
    </div>
    </li>
    <li>
    <div class="p-navEl ">
    <a href="/whats-new/" class="p-navEl-link " data-xf-key="3" data-nav-id="whatsNew">Có gì mới</a>
    </div>
    </li>
    <li>
    <div class="p-navEl ">
    <a href="https://whitehat.vn/video/" class="p-navEl-link " data-xf-key="4" data-nav-id="video">Video</a>
    </div>
    </li>
    <li>
    <div class="p-navEl ">
    <a href="/wargame" class="p-navEl-link " data-xf-key="5" data-nav-id="wargame">Wargame</a>
    </div>
    </li>
    <li>
    <div class="p-navEl ">
    <a href="https://whitehat.vn/" class="p-navEl-link " data-xf-key="6" data-nav-id="tool">Tool</a>
    </div>
    </li>
    <li>
    <div class="p-navEl ">
    <a href="/threads/7078/" class="p-navEl-link " data-xf-key="7" data-nav-id="vinhdanh">Vinh Danh</a>
    </div>
    </li>
    </ul>
    </div>
    </div>
    <div class="p-nav-opposite">
    <div class="p-navgroup p-discovery">
    <a href="/account/bookmarks" class="p-navgroup-link p-navgroup-link--iconic p-navgroup-link--bookmark" aria-label="Dấu trang" title="Dấu trang">
    <img src="/styles/whitehat-2021/icons/Group 8410.png" />
    </a>
    <span class="p-nav-opposite-mobile"><a href="/login/" style="display: inline-block;float: left;margin: 12px 5px;width: 78px;line-height: 36px;background: #d9d9d9;text-align: center;font-size: 13px;border-radius: 5px !important;">Đăng nhập</a></span>
    <a href="/search/" class="p-navgroup-link p-navgroup-link--iconic p-navgroup-link--search" data-xf-click="menu" data-xf-key="/" aria-label="Tìm kiếm" aria-expanded="false" aria-haspopup="true" title="Tìm kiếm">
    <img src="/styles/whitehat-2021/icons/Group 1148.png" />
    </a>
    <div class="menu menu--structural menu--wide" data-menu="menu" aria-hidden="true">
    <form action="/search/search" method="post" class="menu-content" data-xf-init="quick-search">
    <h3 class="menu-header">Tìm kiếm</h3>
    <div class="menu-row">
    <div class="inputGroup inputGroup--joined">
    <input type="text" class="input" name="keywords" placeholder="Tìm kiếm…" aria-label="Tìm kiếm" data-menu-autofocus="true" />
    <select name="constraints" class="js-quickSearch-constraint input" aria-label="Search within">
    <option value>Toàn bộ</option>
    <option value="{&quot;search_type&quot;:&quot;post&quot;}">Chủ đề</option>
    <option value="{&quot;search_type&quot;:&quot;post&quot;,&quot;c&quot;:{&quot;nodes&quot;:[36],&quot;child_nodes&quot;:1}}">Diễn đàn này</option>
    <option value="{&quot;search_type&quot;:&quot;post&quot;,&quot;c&quot;:{&quot;thread&quot;:5281}}">This thread</option>
    </select>
    </div>
    </div>
    <div class="menu-row">
    <label class="iconic"><input type="checkbox" name="c[title_only]" value="1" /><i aria-hidden="true"></i><span class="iconic-label">Chỉ tìm trong tiêu đề
    <span tabindex="0" role="button" data-xf-init="tooltip" data-trigger="hover focus click" title="Tags will also be searched">
    <i class="fa--xf far fa-question-circle u-muted u-smaller" aria-hidden="true"></i>
    </span></span></label>
    </div>
    <div class="menu-row">
    <div class="inputGroup">
    <span class="inputGroup-text" id="ctrl_search_menu_by_member">Bởi:</span>
    <input type="text" class="input" name="c[users]" data-xf-init="auto-complete" placeholder="Thành viên" aria-labelledby="ctrl_search_menu_by_member" />
    </div>
    </div>
    <div class="menu-footer">
    <span class="menu-footer-controls">
    <button type="submit" class="button--primary button button--icon button--icon--search"><span class="button-text">Tìm</span></button>
    <a href="/search/" class="button"><span class="button-text">Tìm nâng cao…</span></a>
    </span>
    </div>
    <input type="hidden" name="_xfToken" value="1723178075,3a6deb8fb7fcf51c78bb639afad7a146" />
    </form>
    </div>
    </div>
    </div>
    </div>
    </nav>
    <div class="p-sectionLinks">
    <div class="p-sectionLinks-inner hScroller" data-xf-init="h-scroller">
    <div class="hScroller-scroll">
    <ul class="p-sectionLinks-list">
    <li>
    <div class="p-navEl ">
    <a href="/whats-new/latest-activity" class="p-navEl-link " data-xf-key="alt+1" data-nav-id="defaultLatestActivity">Hoạt động gần đây</a>
    </div>
    </li>
    <li>
    <div class="p-navEl ">
    <a href="/register/" class="p-navEl-link " data-xf-key="alt+2" data-nav-id="defaultRegister">Đăng ký</a>
    </div>
    </li>
    </ul>
    </div>
    </div>
    </div>
    <div class="offCanvasMenu offCanvasMenu--nav js-headerOffCanvasMenu" data-menu="menu" aria-hidden="true" data-ocm-builder="navigation">
    <div class="offCanvasMenu-backdrop" data-menu-close="true"></div>
    <div class="offCanvasMenu-content">
    <div class="offCanvasMenu-header">
    <img src="/styles/whitehat-2021/logo.svg" srcset alt="WhiteHat.vn" width height />
    <a class="offCanvasMenu-closer" data-menu-close="true" role="button" tabindex="0" aria-label="Đóng"></a>
    </div>
    <div class="p-offCanvasRegisterLink">
    <div class="offCanvasMenu-linkHolder">
    <a href="/login/" class="offCanvasMenu-link" data-xf-click="overlay" data-menu-close="true">
    Đăng nhập
    </a>
    </div>
    <hr class="offCanvasMenu-separator" />
    <div class="offCanvasMenu-linkHolder">
    <a href="/register/" class="offCanvasMenu-link" data-xf-click="overlay" data-menu-close="true">
    Đăng ký
    </a>
    </div>
    <hr class="offCanvasMenu-separator" />
    </div>
    <div class="js-offCanvasNavTarget"></div>
    <div class="canbottom">
    <a href="/threads/1770/" class="p-navgroup-link p-navgroup-link--textual" data-follow-redirects="on">
    <span class="p-navgroup-linkText">Giới thiệu</span>
    </a>
    <a href="/misc/contact" class="p-navgroup-link p-navgroup-link--textual" data-xf-click="overlay" data-follow-redirects="on">
    <span class="p-navgroup-linkText">Liên hệ</span>
    </a>
    </div>
    <h3>CỘNG ĐỒNG AN NINH MẠNG VIỆT NAM</h3>
    <p class="copo">@ 2009 - 2021 Bkav Corporation</p>
    <div class="offCanvasMenu-installBanner js-installPromptContainer" style="display: none;" data-xf-init="install-prompt">
    <div class="offCanvasMenu-installBanner-header">Install the app</div>
    <button type="button" class="js-installPromptButton button"><span class="button-text">Install</span></button>
    </div>
    </div>
    </div>
    <div class="p-body">
    <div class="p-body-inner">

    <div class="p-breadcrumbs-container p-breadcrumbs-container--top">
    <ul class="p-breadcrumbs p-breadcrumbs--top  hide-label" itemscope itemtype="https://schema.org/BreadcrumbList">
    <li itemprop="itemListElement" class="item-1" itemscope itemtype="https://schema.org/ListItem">
    <a href="/categories/thao-luan.20/" itemprop="item">
    <span itemprop="name">Thảo luận</span>
    </a>
    <meta itemprop="position" content="1" />
    </li>
    <li itemprop="itemListElement" class="item-2" itemscope itemtype="https://schema.org/ListItem">
    <a href="/forums/tai-lieu.36/" itemprop="item">
    <span itemprop="name">Tài liệu</span>
    </a>
    <meta itemprop="position" content="2" />
    </li>
    </ul>
    </div>
    <noscript><div class="blockMessage blockMessage--important blockMessage--iconic u-noJsOnly">JavaScript is disabled. For a better experience, please enable JavaScript in your browser before proceeding.</div></noscript>
    <div class="blockMessage blockMessage--important blockMessage--iconic js-browserWarning" style="display: none">You are using an out of date browser. It may not display this or other websites correctly.<br/>You should upgrade or use an <a href="https://www.google.com/chrome/" target="_blank" rel="noopener">alternative browser</a>.</div>
    <div class="p-body-header">
    <div class="p-title ">
    <h1 class="p-title-value">Làm an toàn thông tin thì học gì?</h1>
    </div>
    <div class="p-description">
    <ul class="listInline listInline--bullet">
    <li>
    <i class="fa--xf far fa-user" aria-hidden="true" title="Bắt đầu"></i>
    <span class="u-srOnly">Bắt đầu</span>
    <a href="/members/whf.22/" class="username  u-concealed" dir="auto" itemprop="name" data-user-id="22" data-xf-init="member-tooltip"><span class="username--staff">whf</span></a>
    </li>
    <li>
    <i class="fa--xf far fa-clock" aria-hidden="true" title="Ngày bắt đầu"></i>
    <span class="u-srOnly">Ngày bắt đầu</span>
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/" class="u-concealed"><time class="u-dt" dir="auto" datetime="2015-06-28T23:55:11+0700" data-time="1435510511" data-date-string="28/06/2015" data-time-string="11:55 PM" title="28/06/2015 lúc 11:55 PM">28/06/2015</time></a>
    </li>
    </ul>
    </div>
    </div>

    <div class="p-body-main  ">
    <div class="p-body-contentCol"></div>
    <div class="p-body-content">
    <div class="p-body-pageContent">
    <style>	
        .formButtonGroup-primary > .chevereto-pup-button--XF2 ~ .chevereto-pup-button--XF2 {
            display:none;
        }
        .formRow--fullWidth dd > .chevereto-pup-button--XF2 ~ .chevereto-pup-button--XF2 {
            display:none;
        }
        section.message-responses.js-messageResponses.uw_fcs_comment_section {
            margin-top: 0px;
        }
        .uw_load_prev
        {
            font-weight: normal;
            float:right;
        }
        .message-responseRow.uw-comment-count {
            font-weight: bold;
        }
        .uwcmtbtns
        {
            display:-webkit-box;
        }
        .uw_post_btn_margin
        {
            margin-left: auto;
            margin-right:auto;
        }
        .uw_show_mobile
        {
            display:none !important;
        }
        .reactionsBar.js-reactionsList.uw_rec_logout {
            display: block;
            margin-top: -4px;
        }
        .comment-footer .actionBar-action
        {
            font-size:11px;
        }
        .uw-hidden,.uw_hidden
        {
            display:none;
        }
        .bbCodeBlock-title.uw_quote_thing .bbCodeBlock-sourceJump {
            margin-left: -5px;
        }
        .bbCodeBlock-title.uw_quote_thing {
            background: none;
            display: inline-block;
            padding: 0px;
        }	
        .message-cell.message-cell--main
        {
            flex-direction: column;
            display:flex;
        }
        .message-main.js-quickEditTarget {
            height: auto;
            flex-basis: 100%;
        }
        .uw_comment_edit_box .formRow--fullWidth dd {
            padding: 10px 0px 5px 0px !important;
        }
        .fcs_comment_date
        {
            font-size:11px;
            margin-top: 2px;
        }
        .comment-avatar a
        {
            margin:3px;
            margin-left: 1px;
        }
        .fcs_newIndicator {
            font-size: 10px !important;
        }
        
        .comment-body .js-selectToQuoteEnd {
            height: 0;
            font-size: 0;
            overflow: hidden;
        }
        .js-quickEditTargetComment.js-post {
            margin-top: 4px;
        }
        .formRow--uw_fcs_edit_btns dd {
            float: right;
        }
        .comment-main.is-editing {
            padding-left: 0px !important;
            padding-top: 0px !important;
        }	
        .comment-main {
            padding-left: 0px;
        }
        .uw_fcs_quick_comment .comment-avatar,--.comment-editing .comment-avatar
        {
                display:none !important;
        }	
        @media (max-width: 650px)
        {
            .comment-actionBar.actionBar a {
                margin-left: 4px;
            }
            .uw_fcs_sidebar_cmt,.uw_fcs_quick_comment .comment-avatar,--.comment-editing .comment-avatar
            {
                display:none !important;
            }
            .comment-editing .comment-main.is-editing {
                padding-left: 0px !important; 
            }
            .uw_fcs_quick_comment .comment-main {
                padding-left: 0px !important;
            }
            .uw_show_mobile
            {
                display: inline-block !important;
            }
            .uw_hide_mobile
            {
                display:none !important;
            }
        }
        .comment-footer .actionBar-set.actionBar-set--internal {
            -margin-left: -30px;
        }
        .comment-actionBar .actionBar-set {
            margin-top: 0px;
        }
        .uw_fcs_submit_btn {
            margin-top: 7px;
        }	
        .comment-content {
            margin-left: 13px;
        }
        .comment-contentWrapper {
            margin-bottom: 0px;
        }
        .uw_fcs_quick_comment 
        {
            display:none;
        }
        .comment-main {
            display: block;
            padding-left: 26px;
        }
        .comment-avatar
        {
            height: 24px;
            float:left;
        }
        .uw_fcs_quick_comment .comment-main,.uw_fcs_quick_comment_btn .comment-main
        {
            padding-left: 0px;
        }
        .uw_fcs_quick_comment_btn .uw_fcs_cmt_cancel_btn 
        {
            display:none;
        }
        @media only screen and (min-device-width: 768px) and (max-device-width: 1024px) {
            .message-cell.message-cell--main
            {
                display:block;
            }
            .message-main.js-quickEditTarget
            {
                min-height:100%;
            }
        }
    </style>
    <div class="blockMessage blockMessage--none">
    <div class="shareButtons shareButtons--brandColors shareButtons--iconic" data-xf-init="share-buttons" data-page-url data-page-title data-page-desc data-page-image>
    <span class="shareButtons-label">Chia sẻ:</span>
    <div class="shareButtons-buttons">
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--facebook" data-href="https://www.facebook.com/sharer.php?u={url}">
    <i aria-hidden="true"></i>
    <span>Facebook</span>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--twitter" data-href="https://twitter.com/intent/tweet?url={url}&amp;text={title}">
    <i aria-hidden="true"></i>
    <span>Twitter</span>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--reddit" data-href="https://reddit.com/submit?url={url}&amp;title={title}">
    <i aria-hidden="true"></i>
    <span>Reddit</span>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--pinterest" data-href="https://pinterest.com/pin/create/bookmarklet/?url={url}&amp;description={title}&amp;media={image}">
    <i aria-hidden="true"></i>
    <span>Pinterest</span>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--tumblr" data-href="https://www.tumblr.com/widgets/share/tool?canonicalUrl={url}&amp;title={title}">
    <i aria-hidden="true"></i>
    <span>Tumblr</span>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--whatsApp" data-href="https://api.whatsapp.com/send?text={title}&nbsp;{url}">
    <i aria-hidden="true"></i>
    <span>WhatsApp</span>
    </a>
    <a class="shareButtons-button shareButtons-button--email" data-href="mailto:?subject={title}&amp;body={url}">
    <i aria-hidden="true"></i>
    <span>Email</span>
    </a>
    <a class="shareButtons-button shareButtons-button--share is-hidden" data-xf-init="web-share" data-title data-text data-url data-hide=".shareButtons-button:not(.shareButtons-button--share)">
    <i aria-hidden="true"></i>
    <span>Chia sẻ</span>
    </a>
    <a class="shareButtons-button shareButtons-button--link is-hidden" data-clipboard="{url}">
    <i aria-hidden="true"></i>
    <span>Liên kết</span>
    </a>
    </div>
    </div>
    </div>
    <div class="block-outer-main"><nav class="pageNavWrapper pageNavWrapper--full ">
    <div class="pageNav  ">
    <ul class="pageNav-main">
    <li class="pageNav-page pageNav-page--current "><a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/">1</a></li>
    <li class="pageNav-page "><a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2">2</a></li>
    </ul>
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2" class="pageNav-jump pageNav-jump--next">Tiếp</a>
    </div>
    <div class="pageNavSimple">
    <a class="pageNavSimple-el pageNavSimple-el--current" data-xf-init="tooltip" title="Đi tới trang" data-xf-click="menu" role="button" tabindex="0" aria-expanded="false" aria-haspopup="true">
    1 of 2
    </a>
    <div class="menu menu--pageJump" data-menu="menu" aria-hidden="true">
    <div class="menu-content">
    <h4 class="menu-header">Đi tới trang</h4>
    <div class="menu-row" data-xf-init="page-jump" data-page-url="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-%page%">
    <div class="inputGroup inputGroup--numbers">
    <div class="inputGroup inputGroup--numbers inputNumber" data-xf-init="number-box"><input type="number" pattern="\d*" class="input input--number js-numberBoxTextInput input input--numberNarrow js-pageJumpPage" value="1" min="1" max="2" step="1" required="required" data-menu-autofocus="true" /></div>
    <span class="inputGroup-text"><button type="button" class="js-pageJumpGo button"><span class="button-text">Tới</span></button></span>
    </div>
    </div>
    </div>
    </div>
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2" class="pageNavSimple-el pageNavSimple-el--next">
    Tiếp <i aria-hidden="true"></i>
    </a>
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2" class="pageNavSimple-el pageNavSimple-el--last" data-xf-init="tooltip" title="Cuối">
    <i aria-hidden="true"></i> <span class="u-srOnly">Cuối</span>
    </a>
    </div>
    </nav>
    </div>
    <div class="block block--messages" data-xf-init data-type="post" data-href="/inline-mod/" data-search-target="*">
    <span class="u-anchorTarget" id="posts"></span>
    <div class="block-outer"></div>
    <div class="block-outer js-threadStatusField"></div>
    <div class="block-container lbContainer" data-xf-init="lightbox select-to-quote" data-message-selector=".js-post" data-lb-id="thread-5281" data-lb-universal="1">
    <div class="block-body js-replyNewMessageContainer">
    <article class="message message--post js-post js-inlineModContainer   baidautien " data-author="whf" data-content="post-15138" id="js-post-15138">
    <span class="u-anchorTarget" id="post-15138"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    <section itemscope itemtype="https://schema.org/Person" class="message-user">
    <div class="message-avatar ">
    <div class="message-avatar-wrapper">
    <a href="/members/whf.22/" class="avatar avatar--m" data-user-id="22" data-xf-init="member-tooltip">
    <img src="/data/avatars/m/0/22.jpg?1500608303" srcset="/data/avatars/l/0/22.jpg?1500608303 2x" alt="whf" class="avatar-u22-m" width="96" height="96" loading="lazy" itemprop="image" />
    </a>
    </div>
    </div>
    <div class="message-userDetails">
    <h4 class="message-name"><a href="/members/whf.22/" class="username " dir="auto" itemprop="name" data-user-id="22" data-xf-init="member-tooltip" itemprop="name"><span class="username--style439 username--staff">whf</span></a></h4>
    <h5 class="userTitle message-userTitle" dir="auto" itemprop="jobTitle">Super Moderator</h5>
    <div class="userBanner userBanner--staff message-userBanner" dir="auto" itemprop="jobTitle"><span class="userBanner-before"></span><strong>Thành viên BQT</strong><span class="userBanner-after"></span></div>
    </div>
    <div class="message-userExtras">
    <dl class="pairs pairs--justified register--date">
    <i class="far fa-calendar-alt"></i>
    <span class="register_date">06/07/2013</span>
    </dl>
    <dl class="pairs pairs--justified">
    <i class="far fa-thumbs-up"></i>
    <span>797</span>
    </dl>
    <dl class="pairs pairs--justified baiviet">
    <i class="fal fa-comment-lines"></i>
    <span>1.308 bài viết</span>
    </dl>

    </div>
    <span class="message-userArrow"></span>
    </section>
    </div>
    <div class="message-cell message-cell--main">
    <div class="title" data-xf-init="sticky-header">
    Làm an toàn thông tin thì học gì?
    </div>
    <div class="message-main js-quickEditTarget">
    <div class="shareButtons-sticky" data-xf-init="sticky-header">
    <div class="shareButtons " data-xf-init="share-buttons" data-page-url data-page-title data-page-desc data-page-image>
    <div class="shareButtons-buttons button-firstpost">
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--instagram" data-href="https://www.instagram.com/">
    <i aria-hidden="true"></i>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--facebook" data-href="https://www.facebook.com/sharer.php?u={url}">
    <i aria-hidden="true"></i>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--twitter" data-href="https://twitter.com/intent/tweet?url={url}&amp;text={title}">
    <i aria-hidden="true"></i>
    </a>
    <a class="shareButtons-button shareButtons-button--brand shareButtons-button--bookmark" data-href="/posts/15138/bookmark">
    <i aria-hidden="true"></i>
    </a>
    </div>
    </div>
    </div>
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/whf.22/" class="avatar avatar--xs" data-user-id="22" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/0/22.jpg?1500608303" srcset="/data/avatars/m/0/22.jpg?1500608303 2x" alt="whf" class="avatar-u22-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/whf.22/" class="username " dir="auto" itemprop="name" data-user-id="22" data-xf-init="member-tooltip"><span class="username--style439 username--staff">whf</span></a>
    </li>
    <li class="u-concealed">
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/post-15138" rel="nofollow">
    <time class="u-dt" dir="auto" datetime="2015-06-28T23:55:11+0700" data-time="1435510511" data-date-string="28/06/2015" data-time-string="11:55 PM" title="28/06/2015 lúc 11:55 PM" itemprop="datePublished">28/06/2015</time>
    </a>
    </li>
    <li class="u-concealed">
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/#posts">
    28.878 Lượt xem
    </a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15138" data-lb-caption-desc="whf &middot; 28/06/2015 lúc 11:55 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper"><b><span style="font-size: 14px"><span style="font-family: 'Verdana'">An toàn thông tin là làm gì, nên học những gì? Đó là những câu hỏi mà nhiều bạn dự định theo lĩnh vực an toàn thông tin còn băn khoăn. Do đó trong bài này mình xin chia sẻ bài viết &quot;Làm an toàn thông tin thì học gì?&quot; của tác giả Dương Ngọc Thái - một kỹ sư bảo mật ở Google, đồng tác giả các kĩ thuật tấn công Padding Oracle, BEAST,CRIME.</span></span></b><br/>
    <br/>
    <b>1. Làm an toàn thông tin là làm gì?</b><br/>
    <br/>
    Tôi muốn viết phần này vì nhiều người tưởng tôi làm bảo vệ khi tôi nói tôi làm security. Ngoài ra có lẽ là do thị trường việc làm an toàn thông tin ở Việt Nam không phong phú nên hầu hết đều nghĩ rằng làm an toàn thông tin nghĩa là đảm bảo an toàn hệ thống mạng (network/system security), trong khi thực tế đây chỉ là một trong số rất nhiều công việc trong ngành.<br/>
    <br/>
    Trong bốn phần nhỏ tiếp theo, tôi sẽ giới thiệu bốn nhóm công việc chính trong ngành. Đối với mỗi nhóm công việc, tôi sẽ bàn một chút về triển vọng nghề nghiệp ở Việt Nam và Mỹ, hai nơi mà tôi có dịp được quan sát. Nếu bạn không biết bạn thích làm gì thì cứ chọn một công việc rồi làm thử. Các công việc này đều có liên quan nhau, nên kiến thức mà bạn học được trong quá trình thử vẫn hữu ích cho những nghề khác.<br/>
    <br/>
    <b>1.1. An toàn sản phẩm (product security)</b><br/>
    <br/>
    Công việc chính của nhóm này là làm việc với các đội phát triển sản phẩm để đảm bảo sản phẩm làm ra an toàn cho người dùng và an toàn cho hệ thống của công ty, cụ thể là:<br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul">Kiểm định mã nguồn và thiết kế của sản phẩm</li>
    <li data-xf-list-type="ul">Phát triển các giải pháp kỹ thuật và quy trình phát triển phần mềm an toàn để phát hiện và ngăn chặn những kỹ thuật tấn công đã biết</li>
    <li data-xf-list-type="ul">Đào tạo nhân lực để nâng cao nhận thức về an toàn thông tin cũng như kỹ năng viết mã an toàn</li>
    <li data-xf-list-type="ul">Nghiên cứu các hướng tấn công mới có thể ảnh hưởng hệ thống sản phẩm và dịch vụ của công ty</li>
    </ul>Tóm gọn lại thì nhóm này chuyên tìm lỗ hổng và kỹ thuật tấn công mới. Đây là công việc của tôi và tôi thấy đây là công việc thú vị nhất trong ngành <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" />.<br/>
    <br/>
    Ở Mỹ thì thông thường thì chỉ có các hãng có phần mềm và dịch vụ lớn như Facebook, Google, Microsoft, Oracle, v.v. hay các tập đoàn tài chính ngân hàng lớn mới có đội ngũ tại chỗ để đảm nhiệm công việc này. Các công ty nhỏ thường chỉ thuê dịch vụ của các công ty tư vấn. IBM và Big Four đều có cung cấp dịch vụ tư vấn này. Dẫu vậy nếu được chọn lựa thì tôi sẽ chọn làm cho các công ty chuyên sâu như Matasano, iSec, Leviathan, Gotham, IOActive, Immunity, v.v.<br/>
    <br/>
    Ở Việt Nam thì thị trường việc làm cho người làm an toàn sản phẩm có vẻ ảm đạm hơn. Cho đến nay tôi biết chỉ có một vài công ty ở Việt Nam là có nhân viên chuyên trách lĩnh vực này. Các công ty khác (nếu có quan tâm đến an toàn thông tin) thì hầu như chỉ tập trung vào an toàn vận hành. Các công ty tư vấn an toàn thông tin ở Việt Nam cũng không tư vấn an toàn sản phẩm, mà chỉ tập trung tư vấn chung chung về các quy trình và tiêu chuẩn an toàn thông tin.<br/>
    <br/>
    <b>1.2. An toàn vận hành (operations security)</b><br/>
    <br/>
    Công việc chính của nhóm này là đảm bảo sự an toàn cho toàn bộ hệ thống thông tin của doanh nghiệp, với ba nhiệm vụ chính:<br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul">Ngăn chặn: đưa ra các chính sách, quy định, hướng dẫn về an toàn vận hành; kiện toàn toàn bộ hệ thống thông tin, từ các vành đai cho đến máy tính của người dùng cuối; cấp và thu hồi quyền truy cập hệ thống; quét tìm lỗ hổng trong hệ thống, theo dõi thông tin lỗ hổng mới và làm việc với các bên liên quan để vá lỗi, v.v.</li>
    <li data-xf-list-type="ul">Theo dõi và phát hiện: giám sát an ninh mạng.</li>
    <li data-xf-list-type="ul">Xử lý: phản hồi (incident response) và điều tra số (digital forensics) khi xảy ra sự cố an toàn thông tin, từ tài khoản của nhân viên bị đánh cắp, rò rỉ thông tin sản phẩm mới cho đến tấn công từ chối dịch vụ.</li>
    </ul>Đây là công việc khó nhất, nhưng lại ít phần thưởng nhất của ngành an toàn thông tin.<br/>
    <br/>
    Tương tự như trên, chỉ có các hãng lớn của Mỹ mới có đội ngũ tại chỗ để phụ trách toàn bộ khối lượng công việc đồ sộ này, nhất là mảng xử lý và điều tra. Đa số các công ty chỉ tập trung vào ngăn chặn và sử dụng dịch vụ của bên thứ ba cho hai mảng còn lại. Các hãng như Mandiant, Netwitness hay HBGary cung cấp dịch vụ điều tra các vụ xâm nhập và có rất nhiều hãng khác cung cấp dịch vụ giám sát an ninh mạng.<br/>
    <br/>
    Ở Việt Nam thì thị trường việc làm cho người làm an toàn vận hành tương đối phong phú hơn so với an toàn sản phẩm. Các công ty và tổ chức tài chính lớn đều có một vài vị trí chuyên trách về an toàn vận hành. Đa số người làm về an toàn thông tin ở Việt Nam mà tôi biết là làm trong lĩnh vực này. Dẫu vậy hầu như chưa có ai và công ty tư vấn nào làm về phản hồi và điều tra sự cố.<br/>
    <br/>
    <b>1.3. Phát triển công cụ (applied security)</b><br/>
    <br/>
    Công việc chính của nhóm này là phát triển và cung cấp các công cụ, dịch vụ và thư viện phần mềm có liên quan đến an toàn thông tin cho các nhóm phát triển sản phẩm sử dụng lại.<br/>
    <br/>
    Nhóm này bao gồm các kỹ sư nhiều năm kinh nghiệm và có kiến thức vững chắc về an toàn thông tin, viết mã an toàn và mật mã học. Họ phát triển các thư viện và dịch vụ dùng chung như phân tích mã tĩnh - phân tích mã động (static - dynamic code analysis), hộp cát (sandboxing), xác thực (authentication), kiểm soát truy cập (authorization), mã hóa (encryption) và quản lý khóa (key management), v.v.<br/>
    <br/>
    Đây là dạng công việc dành cho những ai đang viết phần mềm chuyên nghiệp và muốn chuyển qua làm về an toàn thông tin. Đây cũng là công việc của những người thích làm an toàn sản phẩm nhưng muốn tập trung vào việc xây dựng sản phẩm hơn là tìm lỗ hổng.<br/>
    <br/>
    Rõ ràng loại công việc này chỉ xuất hiện ở các công ty phần mềm lớn. Ở các công ty phần mềm nhỏ hơn thì các kỹ sư phần mềm thường phải tự cáng đáng công việc này mà ít có sự hỗ trợ từ nguồn nào khác. Ở Việt Nam thì tôi không biết có ai làm dạng công việc này không.<br/>
    <br/>
    <b>1.4. Tìm diệt mã độc và các nguy cơ khác (threat analysis)</b><br/>
    <br/>
    Ngoài an toàn sản phẩm ra thì đây là một lĩnh vực mà tôi muốn làm. Công việc chính của nhóm này là phân tích, truy tìm nguồn gốc và tiêu diệt tận gốc mã độc và các tấn công có chủ đích (targeted attack). Mã độc ở đây có thể là virút, sâu máy tính, hay mã khai thác các lỗ hổng đã biết hoặc chưa được biết đến mà phần mềm diệt virút thông thường chưa phát hiện được. Các loại mã độc này thường được sử dụng trong các tấn công có chủ đích vào doanh nghiệp.<br/>
    <br/>
    Tôi nghĩ rằng sau hàng loạt vụ tấn công vừa rồi thì chắc hẳn các công ty lớn với nhiều tài sản trí tuệ giá trị đều muốn có những chuyên gia trong lĩnh vực này trong đội ngũ của họ. Ngoài ra các công ty chuyên về điều tra và xử lý sự cố như Mandiant, HBGary hay Netwitness mà tôi đề cập ở trên đều đang ăn nên làm ra và lúc nào cũng cần người. Các công ty sản xuất phần mềm diệt virút dĩ nhiên cũng là một lựa chọn.<br/>
    <br/>
    Ở Việt Nam thì tôi nghĩ hầu hết doanh nghiệp vẫn chưa thấy được nguy cơ đến từ các cuộc tấn công có chủ đích, thành ra họ sẽ không tuyển người chuyên trách vấn đề này. Tôi cũng không biết có công ty tư vấn nào ở Việt Nam chuyên về điều tra và xử lý sự cố hay không. Tôi nghĩ lựa chọn khả dĩ nhất cho những người thích mảng công việc này là các công ty phần mềm diệt virút.<br/>
    <br/>
    Tuy nhiên cũng cần lưu ý rằng trong vài năm gần đây ở Việt Nam còn xuất hiện những loại mã độc nhắm vào đông đảo người dùng máy tính bình thường. Vấn nạn này có lẽ sẽ còn kéo dài trong nhiều năm tới và lẽ đương nhiên &quot;phe ta&quot; lúc nào cũng cần thêm những chiến sĩ lành nghề như anh TQN. Thành ra dẫu triển vọng nghề nghiệp không sáng sủa cho lắm, nhưng tôi rất hi vọng sẽ ngày càng nhiều người tham gia vào việc phân tích các mã độc nhắm vào người dùng máy tính ở Việt Nam. Đối với tôi họ là những người hùng thầm lặng, chiến đấu đêm ngày với các &quot;thế lực thù địch&quot; để bảo vệ tất cả chúng ta.<br/>
    <br/>
    <b>2. Học như thế nào?</b><br/>
    <br/>
    Đa số những bạn viết thư cho tôi đều đang học đại học ngành CNTT và tất cả đều than rằng chương trình học quá chán, không có những thứ mà các bạn muốn học. Tôi nghĩ đây là một ngộ nhận.<br/>
    <br/>
    Hối tiếc lớn thứ nhì trong sự nghiệp học tập mấy chục năm của tôi là đã không học nghiêm túc khi còn là sinh viên (hối tiếc lớn nhất là tôi đã không nghỉ hẳn, nhưng đó là một câu chuyện dài khác). Tôi cũng đã nghĩ rằng chương trình học ở đại học là lạc hậu và không cần thiết. Bây giờ nhìn lại thì tôi thấy nội dung và cách dạy của từng môn học thì đúng là lạc hậu , nhưng toàn bộ giáo trình đại học vẫn cung cấp được một cái sườn kiến thức rất cần thiết cho một kỹ sư an toàn thông tin.<br/>
    <br/>
    Ở đại học người ta có cách tiếp cận top-down, nghĩa là dạy từ đầu đến cuối những kiến thức nằm trong chương trình. Điều này dễ dẫn đến tình trạng là người học phải học những kiến thức mà họ không thấy cần thiết. Nếu chương trình học cũ kỹ và không có nhiều thực hành, hoặc người dạy không chỉ ra được bức tranh toàn cảnh, vị trí hiện tại của người học và bước tiếp theo họ nên làm là gì thì người học sẽ dễ cảm thấy rằng họ đang phí thời gian học những kiến thức vô bổ.<br/>
    <br/>
    Trong khi khi đi làm thì cách tiếp cận là bottom-up, nghĩa là lao vào làm, thấy thiếu kiến thức chỗ nào thì học để bù vào chỗ đó. Lúc này tôi hoàn toàn chủ động trong việc học và tôi cũng hiểu rõ tôi cần học cái gì và tại sao. Điều thú vị là mỗi khi truy ngược lại nguồn gốc của những kiến thức tôi cần phải có, tôi thường thấy chúng nằm trong chương trình đại học.<br/>
    <br/>
    Ví dụ như tôi muốn luyện kỹ năng dịch ngược mã phần mềm (reverse code engineering - RCE) thì tôi thấy rằng tôi cần phải có kiến thức về tổ chức và cấu trúc máy tính. Hoặc nếu tôi muốn học về mật mã học thì tôi phải học lý thuyết tính toán, mà khởi nguồn là lý thuyết automata. Nhưng tại sao trước đó tôi cũng đi làm nhưng không thấy được những lỗ hổng kiến thức này? Tôi nghĩ là do tôi làm không đủ sâu. Ví dụ như nếu bạn suốt ngày chỉ lập trình PHP thì bạn sẽ không thể hiểu được tại sao phải nắm vững tổ chức và kiến trúc máy tính. Hoặc giả như công việc của bạn là sysadmin thì cũng sẽ rất khó để bạn thấy được tại sao cần phải học lý thuyết automata.<br/>
    <br/>
    Những gì tôi nói lan man ở trên có thể tóm gọn lại thế này:<br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul">Học dựa theo chương trình đại học. Nếu bạn đang học đại học các ngành công nghệ thông tin, khoa học máy tính hay toán tin thì nên tập trung vào việc học các môn trong trường. Các học liệu trong phần 3 cũng được soạn theo các đại học lớn trên thế giới.</li>
    <li data-xf-list-type="ul">Học kiến thức căn bản thật vững (cái gì là căn bản thì xem phần 3), những món còn lại khi nào cần (căn cứ vào nhu cầu công việc) thì hẵng học.</li>
    <li data-xf-list-type="ul">Tìm dự án lề (side project) mà bạn thích để làm để có thể nhanh chóng nhận ra những mảng kiến thức còn thiếu.</li>
    <li data-xf-list-type="ul">Thời điểm tốt nhất để học một cái gì đó là khi bạn đang là sinh viên. Thời điểm tốt thứ hai là ngay bây giờ!</li>
    </ul><br/>
    Các lớp mà tôi liệt kê trong phần 3 đa số là của đại học Stanford. Bạn không cần phải đến tận nơi, ngồi trong lớp mới có thể học được. Tôi thấy trong nhiều trường hợp thì bạn chỉ cần đọc lecture notes, sách giáo khoa mà lớp sử dụng rồi làm bài tập đầy đủ thì vẫn sẽ tiếp thu đủ kiến thức. Một số lớp mà tôi liệt kê dưới đây được dạy miễn phí rộng rãi trên <a href="https://www.coursera.org/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fwww.coursera.org%2F&amp;hash=8fdcd6bc6f684c70f61e08dd55a89192" rel="noopener">Coursera</a>.<br/>
    <br/>
    Bây giờ chỉ cần bạn chịu học thì muốn học cái gì cũng có lớp và học liệu miễn phí. Nhưng mà học cái gì bây giờ?<br/>
    <br/>
    <b>3. Học cái gì?</b><br/>
    <br/>
    Có ba món quan trọng cần phải học: lập trình, lập trình và lập trình! Để làm việc được trong ngành này, bạn phải yêu thích lập trình. Không có cách nào khác. Thề luôn!<br/>
    <br/>
    Tôi dành khá nhiều thời gian tìm hiểu giáo trình khoa học máy tính của các trường đại học lớn trên thế giới và tôi thấy tất cả các môn học đều có phần bài tập là lập trình. Học cái gì viết phần mềm cho cái đó. Học về hệ điều hành thì phần bài tập là viết một hệ điều hành. Học về mạng thì viết phần mềm giả lập router, switch hay firewall. Cá nhân tôi cũng thấy rằng lập trình là cách tốt nhất để tiếp thu kiến thức một môn học nào đó, biến nó thành của mình. Nói cách khác, lập trình là một cách mã hóa tri thức khá hiệu quả.<br/>
    <br/>
    Ngoài ra nhìn vào mô tả công việc ở phần 1, bạn cũng có thể thấy kỹ năng lập trình quan trọng đến dường nào, bởi hầu hết các vấn đề và giải pháp của an toàn thông tin là đến từ phần mềm. Rõ ràng muốn tìm lỗi của phần mềm thì bạn phải hiểu được phần mềm thông qua mã nguồn trực tiếp hay trung gian của nó. Rất có thể bạn sẽ không phải lập trình hàng ngày, nhưng bạn phải viết được những công cụ nhỏ hay những thư viện hỗ trợ cho công việc và các lập trình viên khác.<br/>
    <br/>
    Vậy làm thế nào để lập trình giỏi? Câu hỏi này làm tôi nhớ đến câu chuyện cười về ông lập trình viên không thể ra khỏi phòng tắm vì trên chai dầu gội có ghi hướng dẫn sử dụng là &quot;cho vào tay, xoa lên đầu, xả nước và lập lại&quot;. Từ khóa trong câu chuyện này là &quot;lập lại&quot;: muốn giỏi lập trình thì cách tốt nhất là lập trình nhiều vô!<br/>
    <br/>
    Nhưng mà lập trình bằng ngôn ngữ gì bây giờ? Đây là câu hỏi dễ làm cho các lập trình viên oánh nhau nhất ;-). Cá nhân tôi thấy rằng người làm an toàn thông tin bây giờ cần phải thông thạo C, x86 Assembly, Python (hoặc Ruby) và JavaScript. Tôi có nói lý do tại sao trong phần giới thiệu sách tiếp theo.<br/>
    <br/>
    <b>Lập trình</b><br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul"><a href="http://www2.cs.uregina.ca/~hilder/cs833/Other%20Reference%20Materials/The%20C%20Programming%20Language.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww2.cs.uregina.ca%2F%7Ehilder%2Fcs833%2FOther%2520Reference%2520Materials%2FThe%2520C%2520Programming%2520Language.pdf&amp;hash=20fb1434379f6c143c08a8d23067ac8b" rel="noopener">Brian Kernighan, Dennis Ritchie, The C Programming Language (2nd Edition)</a>: kinh điển và phải-đọc cho tất cả những ai muốn học C! Linus Torvalds từng nói rằng &quot;[...] all right-thinking people know that (a) K&amp;R are _right_ and (b) K&amp;R are right&quot;. Tôi đã từng rất sợ C (vì nghĩ nó phức tạp), và cuốn này giúp tôi không còn sợ nữa.</li>
    <li data-xf-list-type="ul"><a href="http://guanzhou.pub/files/Computer%20System_EN.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fguanzhou.pub%2Ffiles%2FComputer%2520System_EN.pdf&amp;hash=a4d471ab800660daa0249a94d9cf9049" rel="noopener">Randal Bryant, David O'Hallaron, Computer Systems: A Programmer's Perspective</a>: cuốn này được dùng cho lớp CS107. Đọc cuốn này và làm bài tập của lớp CS107 sẽ rèn cho bạn kỹ năng lập trình C và x86 Assembly. Sau khi đọc cuốn này, bạn sẽ biết tại sao có lỗi tràn bộ đệm và cách khai thác chúng. Tôi rất thích các chương nói về x86 và sự liên kết giữa các công cụ như preprocessor, compiler và linker.</li>
    <li data-xf-list-type="ul"><a href="https://cii.s3.amazonaws.com/book/pdf/quickref.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fcii.s3.amazonaws.com%2Fbook%2Fpdf%2Fquickref.pdf&amp;hash=f1e6b6b10431cca587b3ea2a4381a061" rel="noopener">David Hanson, C Interfaces and Implementations</a>: muốn mau &quot;lên cơ&quot; bida thì phải thường xuyên xem người khác chơi để mà học &quot;đường&quot; mới. Tương tự, muốn giỏi lập trình thì phải thường xuyên đọc mã của những cao thủ. David Hanson là một cao thủ C và cuốn sách này sẽ chỉ cho bạn nhiều &quot;đường&quot; mới trong việc sử dụng C. Tôi thích các bài tập của cuốn sách này. Tôi nghĩ chỉ cần luyện các bài này là đủ để trở thành một lập trình viên C hạng lông.</li>
    <li data-xf-list-type="ul"><a href="https://repo.zenk-security.com/Magazine%20E-book/Gray%20Hat%20Python%20-%20Python%20Programming%20for%20Hackers%20and%20Reverse%20Engineers%20(2009).pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Frepo.zenk-security.com%2FMagazine%2520E-book%2FGray%2520Hat%2520Python%2520-%2520Python%2520Programming%2520for%2520Hackers%2520and%2520Reverse%2520Engineers%2520%282009%29.pdf&amp;hash=860f449734b892ec3f4ee37b5d351522" rel="noopener">Justin Seitz, Gray Hat Python</a>: Python Programming for Hackers and Reverse Engineers: cuốn này sẽ giúp bạn sử dụng Python để viết những công cụ nho nhỏ mà bất kỳ ai làm an toàn thông tin cũng sẽ phải viết một vài lần trong đời.</li>
    <li data-xf-list-type="ul"><span style="color: #0000ff"><a href="https://andersonguelphjs.github.io/OReilly_JavaScript_The_Good_Parts_May_2008.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fandersonguelphjs.github.io%2FOReilly_JavaScript_The_Good_Parts_May_2008.pdf&amp;hash=eba6c14851b32146c71f3f6f6c9d921b" rel="noopener">Douglas Crockford, JavaScript: The Good Parts</a></span>: JavaScript là ngôn ngữ thống trị WWW. Nếu bạn muốn làm an toàn (ứng dụng và trình duyệt) web thì bắt buộc phải thành thạo ngôn ngữ. Cuốn sách rất mỏng này của tác giả JSON giới thiệu đầy đủ những vấn đề mà người làm an toàn ứng dụng cần phải biết về JavaScript. Cuốn này có thể dùng làm sách giáo khoa thay cho cuốn &quot;Javascript: The Definitive Guide&quot; trong lớp CS142 (xem bên dưới). Đọc cuốn này tôi mới hiểu closure là gì và bản chất prototypal của JavaScript.</li>
    </ul><b>Hệ điều hành</b><br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul"><a href="http://iit.qau.edu.pk/books/OS_8th_Edition.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fiit.qau.edu.pk%2Fbooks%2FOS_8th_Edition.pdf&amp;hash=43a6f01a37cfc7f59d94046985b34e3a" rel="noopener">Abraham Silberschatz, Peter Galvin, and Greg Gagne, Operating System Concepts, 8th Edition Update</a>: cuốn này là giáo trình của lớp CS140. Tôi nghĩ không cần đọc cuốn này, chỉ cần đọc notes và làm bài tập (viết các phần khác nhau của một hệ điều hành!) là đủ. Đây là một lớp nặng. Tôi theo đuổi lớp CS140 này giữa chừng thì phải dừng lại do không có đủ thời gian.</li>
    <li data-xf-list-type="ul"><a href="http://www.intel.com/content/www/us/en/processors/architectures-software-developer-manuals.html" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.intel.com%2Fcontent%2Fwww%2Fus%2Fen%2Fprocessors%2Farchitectures-software-developer-manuals.html&amp;hash=8dfc9b77bc000439b722413ca95e37b6" rel="noopener">Intel Software Developer Manuals</a>: tôi thấy nên đọc tài liệu của 80386 trước, rồi sau đó hẵng đọc tài liệu của các <a href="http://www.intel.com/content/www/us/en/processors/architectures-software-developer-manuals.html" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.intel.com%2Fcontent%2Fwww%2Fus%2Fen%2Fprocessors%2Farchitectures-software-developer-manuals.html&amp;hash=8dfc9b77bc000439b722413ca95e37b6" rel="noopener">CPU mới hơn</a>.</li>
    <li data-xf-list-type="ul"><a href="https://access.redhat.com/documentation/en-US/Red_Hat_Enterprise_Linux/4/html/Introduction_To_System_Administration/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Faccess.redhat.com%2Fdocumentation%2Fen-US%2FRed_Hat_Enterprise_Linux%2F4%2Fhtml%2FIntroduction_To_System_Administration%2F&amp;hash=2aa99c6d92970b875f4669f587b73f96" rel="noopener">Red Hat, Introduction to System Administration</a>: tôi rất thích chương nói về &quot;philosophy of sysadmin&quot; của cuốn này và tôi nghĩ kỹ năng quản trị hệ thống là cực kỳ cần thiết khi muốn nghiên cứu các kỹ thuật tấn công/phòng thủ mới. Không thể làm an toàn vận hành nếu không có kỹ năng quản trị hệ thống.</li>
    <li data-xf-list-type="ul">Sẽ đọc: Mark Russinovich, David Solomon, Alex Ionescu, Windows Internals, Part 1: Covering Windows Server 2008 R2 and Windows 7.</li>
    </ul><b>Mạng máy tính</b><br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul"><a href="http://ftp://167.205.50.45/pub/ISO-IMAGES/linux/filenya-putu-shinoda/bukulinux/the-protocols-tcp-ip-illustrated-volume-1.9780201633467.24290.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fftp%3A%2F%2F167.205.50.45%2Fpub%2FISO-IMAGES%2Flinux%2Ffilenya-putu-shinoda%2Fbukulinux%2Fthe-protocols-tcp-ip-illustrated-volume-1.9780201633467.24290.pdf&amp;hash=be9d65c41d4c5590234c0ae6b23456b4" rel="noopener">Richard Stevens, TCP/IP Illustrated Vol I</a>: cuốn sách này quá nổi tiếng rồi nên tôi nghĩ không cần phải giới thiệu. Tôi chưa đọc Vol II, III nhưng nhất định sẽ tìm đọc trong thời gian tới. Lớp CS144 dùng một cuốn sách khác. Tôi chưa học lớp này, nhưng tôi thấy bài tập của họ khá thú vị.</li>
    <li data-xf-list-type="ul"><a href="http://data.vnuhcm.edu.vn/data/dulieu/khcn/luutru/(Sams)%20Inside%20Network%20Perimeter%20Security,%202nd%20Edition%20(2005)%20-%20Lib.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fdata.vnuhcm.edu.vn%2Fdata%2Fdulieu%2Fkhcn%2Fluutru%2F%28Sams%29%2520Inside%2520Network%2520Perimeter%2520Security%2C%25202nd%2520Edition%2520%282005%29%2520-%2520Lib.pdf&amp;hash=baab513423f598fe4b46265978df9c8f" rel="noopener">Stephen Northcutt, Lenny Zeltser, Scott Winters, Karen Kent, Ronald W. Ritchey, Inside Network Perimeter Security, 2nd Edition</a>: tôi thích cuốn này vì nó viết rất dễ hiểu về các vấn đề và công cụ thường gặp trong an toàn mạng.</li>
    <li data-xf-list-type="ul">Sẽ đọc: Fyodor, Nmap Network Scanning.</li>
    </ul>Sau khi đã có những kiến thức cơ bản ở trên, bạn có thể theo đuổi lớp CS155. Lớp này có trên Coursera với tên Computer Security.<br/>
    <br/>
    <b>Tìm lỗi phầm mềm</b><br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul"><a href="https://books.google.com.vn/books/about/The_Art_of_Software_Security_Assessment.html?id=t2yA8vtfxDsC&amp;redir_esc=y" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fbooks.google.com.vn%2Fbooks%2Fabout%2FThe_Art_of_Software_Security_Assessment.html%3Fid%3Dt2yA8vtfxDsC%26redir_esc%3Dy&amp;hash=7525f5e58161f8a530f9612a0b76d733" rel="noopener">Mark Dowd, John McDonald, Justin Schuh, The Art of Software Security Assessment</a>: Identifying and Preventing Software Vulnerabilities: Kinh điển và phải-đọc! Cuốn này là kinh thánh của lĩnh vực an ninh ứng dụng. Tôi thích nhất phần nói về tràn số nguyên và những vấn đề của ngôn ngữ C trong cuốn này.</li>
    <li data-xf-list-type="ul"><a href="https://leaksource.files.wordpress.com/2014/08/the-web-application-hackers-handbook.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fleaksource.files.wordpress.com%2F2014%2F08%2Fthe-web-application-hackers-handbook.pdf&amp;hash=6ce652dd342740d0702087947e56604d" rel="noopener">Dafydd Stuttard, Marcus Pinto, The Web Application Hacker's Handbook</a>: Discovering and Exploiting Security Flaws: cuốn này tập trung vào ứng dụng web. Tôi không đọc cuốn này kỹ lắm, mà chỉ thường dùng nó để tham khảo. Dẫu vậy tôi nghĩ nó là một cuốn giới thiệu tốt cho những ai mới bắt đầu.</li>
    <li data-xf-list-type="ul"><a href="http://www.mediafire.com/download/323nod3n038aast/The+tangled+web.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.mediafire.com%2Fdownload%2F323nod3n038aast%2FThe%2Btangled%2Bweb.pdf&amp;hash=31936ff4ec1f271f6f6d3bd3bcc9116e" rel="noopener">Michal Zalewski, The Tangled Web</a>: cuốn này mới xuất bản gần đây nhưng đã ngay lập tức trở thành kinh điển! Cuốn này đúc kết quá trình nghiên cứu về an ninh web trong vài năm trời của một trong những hacker xuất sắc nhất thế giới. Tôi nghĩ chỉ cần đọc cuốn này là bạn đã có thể bắt đầu tìm lỗ kiếm tiền được rồi. Cuốn này và cuốn ở trên được dùng làm sách giáo khoa của lớp CS142.</li>
    <li data-xf-list-type="ul">Sẽ đọc: Tobias Klein, A Bug Hunter's Diary: A Guided Tour Through the Wilds of Software Security</li>
    </ul><b>Dịch ngược mã phần mềm</b><br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul"><a href="http://www.foo.be/cours/dess-20122013/b/Eldad_Eilam-Reversing__Secrets_of_Reverse_Engineering-Wiley(2005).pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.foo.be%2Fcours%2Fdess-20122013%2Fb%2FEldad_Eilam-Reversing__Secrets_of_Reverse_Engineering-Wiley%282005%29.pdf&amp;hash=e7f50bfaa59d42325287cc8db5121baf" rel="noopener">Eldad Eilam, Reversing: Secrets of Reverse Engineering</a>: mặc dù có rất nhiều người viết về RCE nhưng tôi thấy đây là cuốn duy nhất hệ thống hóa được các bước quan trọng cần phải làm khi cần dịch ngược mã của một tệp chương trình nào đó.</li>
    <li data-xf-list-type="ul"><a href="http://www.mediafire.com/download/10vc5o1jofk84u8/No.Starch.The.IDA.Pro.Book.2nd.Edition.Jun.2011.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.mediafire.com%2Fdownload%2F10vc5o1jofk84u8%2FNo.Starch.The.IDA.Pro.Book.2nd.Edition.Jun.2011.pdf&amp;hash=da7acbf7804200d943540d39be9f8cc9" rel="noopener">Chris Eagle, The IDA Pro Book: The Unofficial Guide to the World's Most Popular Disassembler</a>: IDA Pro là công cụ tốt nhất để làm RCE và đây là cuốn sách tốt nhất về IDA Pro. Nắm vững C và x86 Assembly thì chỉ cần đọc cuốn này là bạn có thể bắt đầu RCE các phần mềm phức tạp.</li>
    <li data-xf-list-type="ul">Tham khảo các tài liệu về dịch ngược mã phần mềm của lớp PenTest của đại học NYU.</li>
    <li data-xf-list-type="ul">Sẽ đọc: Christian Collberg, Surreptitious Software: Obfuscation, Watermarking, and Tamperproofing for Software Protection: Obfuscation, Watermarking, and Tamperproofing for Software Protection</li>
    <li data-xf-list-type="ul">Sẽ đọc: Michael Sikorski, Andrew Honig, Practical Malware Analysis: The Hands-On Guide to Dissecting Malicious Software</li>
    </ul><b>Điều tra số (digital forensics)</b><br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul"><a href="http://sergiob.org/unam/DGSCA/forense/FileSystemAnalysis.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fsergiob.org%2Funam%2FDGSCA%2Fforense%2FFileSystemAnalysis.pdf&amp;hash=0cece9df6f5e938f2e83aa7cafb3eeb0" rel="noopener">Brian Carrier, File System Forensic Analysis</a>: Brian Carrier là tác giả của bộ công cụ forensic nổi tiếng <a href="http://www.sleuthkit.org/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.sleuthkit.org%2F&amp;hash=24cebc767eab7278d31a5ae318fa0ccd" rel="noopener">The Sleuth Kit</a>. Cuốn này đã giúp tôi &quot;khai quật&quot; được một đoạn video bị xóa lưu trong một máy camera quay lén các máy ATM.</li>
    <li data-xf-list-type="ul">Sẽ đọc: Cory Altheide, Harlan Carvey, Digital Forensics with Open Source Tools</li>
    </ul><b>Mật mã hóa</b><br/>
    <br/>
    <ul>
    <li data-xf-list-type="ul"><a href="http://cdn.preterhuman.net/texts/cryptography/Practical%20Cryptography.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fcdn.preterhuman.net%2Ftexts%2Fcryptography%2FPractical%2520Cryptography.pdf&amp;hash=73e121b7e699940ed2af7fbd2d09f44f" rel="noopener">Niels Ferguson, Bruce Schneier, Practical Cryptography</a>: tôi có nhiều kỷ niệm đẹp với cuốn này ;-). Hầu hết các kết quả làm việc của tôi trong vài năm vừa rồi là nhờ vào việc đọc cuốn này. Tôi chép lại đây giới thiệu rất hay của một người bạn: &quot;The best security books, you can read &quot;inside out&quot;, taking any recommendation on what to do and looking for people to do the opposite to find flaws. &quot;Firewalls and Internet Security&quot; was like that. So was &quot;Practical Unix Security&quot;, and so is TOASSA. This is that book for crypto. It's also the one book on crypto you should allow yourself to read until you start actually finding crypto flaws.&quot;</li>
    <li data-xf-list-type="ul">Jonathan Katz, <a href="https://www.u-cursos.cl/usuario/777719ab2ddbbdb16d99df29431d3036/mi_blog/r/1_book-introduction_to_modern_cryptography.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fwww.u-cursos.cl%2Fusuario%2F777719ab2ddbbdb16d99df29431d3036%2Fmi_blog%2Fr%2F1_book-introduction_to_modern_cryptography.pdf&amp;hash=5b76acc0335bc771991dadb63626dd02" rel="noopener">Yehuda Lindell, Introduction to Modern Cryptography</a>: Principles and Protocols: đây là sách giáo khoa của lớp CS255. Lớp này là lớp <a href="https://www.coursera.org/course/crypto" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fwww.coursera.org%2Fcourse%2Fcrypto&amp;hash=02d514f6fd2e344fba9ebb3197a4afec" rel="noopener">Cryptography</a> trên Coursera.</li>
    <li data-xf-list-type="ul">Sẽ đọc: Adam Young, Moti Yung, Malicious Cryptography: Exposing Cryptovirolog</li>
    </ul><br/>
    Chú ý đây là những cuốn sách tập trung vào công việc hàng ngày và sở thích của tôi -- nói cách khác, còn thiếu nhiều sách của các mảng công việc khác. Dẫu vậy tôi nghĩ những cuốn sách này sẽ giúp bạn có được một kiến thức nền tảng vững chắc để từ đó theo đuổi các nghề nghiệp khác nhau trong ngành an toàn thông tin.<br/>
    <br/>
    Ngoài ra trong các sách mà tôi vừa liệt kê không có cuốn sách toán (và lý thuyết khoa học máy tính) nào cả. Tôi nghĩ bạn sẽ tự có câu trả lời cho câu hỏi &quot;Có nên học toán hay không?&quot; khi bắt đầu học mật mã. Về hai mảng này thì tôi rất thích lớp &quot;Great Ideas in Theoretical Computer Science&quot; của <a href="http://www.scottaaronson.com/blog/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.scottaaronson.com%2Fblog%2F&amp;hash=b7ac81d887937a3fe3ac1a091bf2a75e" rel="noopener">Scott Aaronson</a> và cuốn &quot;<a href="http://shoup.net/ntb/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fshoup.net%2Fntb%2F&amp;hash=44ebd3361ad8689b5edb33ae9727795c" rel="noopener">A Computational Introduction to Number Theory and Algebra</a>&quot; của Victor Shoup. Thích đến nỗi tôi phải viết đoạn này chỉ để nhắc đến chúng ;-). Tôi cũng đã từng dành ra nhiều tháng để đánh vật với <a href="http://www-math.mit.edu/~sipser/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww-math.mit.edu%2F%7Esipser%2F&amp;hash=359d572ba11ba2aab3b6f19baca5760e" rel="noopener">Introduction to the Theory of Computation</a> của Michael Sipser. Nhưng thôi, tôi không muốn giới thiệu sách toán nữa vì tôi rất dốt món này!<br/>
    <div style="text-align: center">---&#8203;</div></div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList is-active ">
    <ul class="reactionSummary">
    <li><span class="reaction reaction--small reaction--1" data-reaction-id="1"><i aria-hidden="true"></i><img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="reaction-sprite js-reaction" alt="Thích" title="Thích" /></span></li>
    </ul>
    <span class="u-srOnly">Reactions:</span>
    <a class="reactionsBar-link" href="/posts/15138/reactions" data-xf-click="overlay" data-cache="false"><bdi>Fox_LB</bdi>, <bdi>LB - Khanh</bdi>, <bdi>lapduc</bdi> and 3 others</a>
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="tpcntt" data-content="post-15139" id="js-post-15139">
    <span class="u-anchorTarget" id="post-15139"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/tpcntt.54446/" class="avatar avatar--xs avatar--default avatar--default--dynamic" data-user-id="54446" data-xf-init="member-tooltip" style="background-color: #cc6666; color: #602020">
    <span class="avatar-u54446-s">T</span>
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/tpcntt.54446/" class="username " dir="auto" itemprop="name" data-user-id="54446" data-xf-init="member-tooltip"><span class="username--style2">tpcntt</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15139" data-lb-caption-desc="tpcntt &middot; 17/07/2015 lúc 11:18 AM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    Bạn có thể share những tài liệu mà bạn gợi ý nên đọc kia không <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite5" alt=":confused:" title="Confused    :confused:" loading="lazy" data-shortname=":confused:" /></div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList is-active ">
    <ul class="reactionSummary">
    <li><span class="reaction reaction--small reaction--1" data-reaction-id="1"><i aria-hidden="true"></i><img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="reaction-sprite js-reaction" alt="Thích" title="Thích" /></span></li>
    </ul>
    <span class="u-srOnly">Reactions:</span>
    <a class="reactionsBar-link" href="/posts/15139/reactions" data-xf-click="overlay" data-cache="false"><bdi>mr-invad3r</bdi> and <bdi>chunhattien</bdi></a>
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15139">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="whf" data-content="post-15140" id="js-post-15140">
    <span class="u-anchorTarget" id="post-15140"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/whf.22/" class="avatar avatar--xs" data-user-id="22" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/0/22.jpg?1500608303" srcset="/data/avatars/m/0/22.jpg?1500608303 2x" alt="whf" class="avatar-u22-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/whf.22/" class="username " dir="auto" itemprop="name" data-user-id="22" data-xf-init="member-tooltip"><span class="username--style439 username--staff">whf</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15140" data-lb-caption-desc="whf &middot; 17/07/2015 lúc 12:30 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    <blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch">
    <div class="bbCodeBlock-title">
    tpcntt;26717 đã viết:
    </div>
    <div class="bbCodeBlock-content">
    <div class="bbCodeBlock-expandContent js-expandContent ">
    Bạn có thể share những tài liệu mà bạn gợi ý nên đọc kia không <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite5" alt=":confused:" title="Confused    :confused:" loading="lazy" data-shortname=":confused:" />
    </div>
    <div class="bbCodeBlock-expandLink js-expandLink"><a role="button" tabindex="0">Nhấn để mở rộng...</a></div>
    </div>
    </blockquote><br/>
    Mình sẽ sớm cập nhật link tải những tài liệu trên nhé <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite9" alt=":eek:" title="Eek!    :eek:" loading="lazy" data-shortname=":eek:" /></div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList is-active ">
    <ul class="reactionSummary">
    <li><span class="reaction reaction--small reaction--1" data-reaction-id="1"><i aria-hidden="true"></i><img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="reaction-sprite js-reaction" alt="Thích" title="Thích" /></span></li>
    </ul>
    <span class="u-srOnly">Reactions:</span>
    <a class="reactionsBar-link" href="/posts/15140/reactions" data-xf-click="overlay" data-cache="false"><bdi>newbielearner123</bdi></a>
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15140">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="madawg" data-content="post-15141" id="js-post-15141">
    <span class="u-anchorTarget" id="post-15141"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/madawg.1217/" class="avatar avatar--xs avatar--default avatar--default--dynamic" data-user-id="1217" data-xf-init="member-tooltip" style="background-color: #333399; color: #9f9fdf">
    <span class="avatar-u1217-s">M</span>
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/madawg.1217/" class="username " dir="auto" itemprop="name" data-user-id="1217" data-xf-init="member-tooltip"><span class="username--style2">madawg</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15141" data-lb-caption-desc="madawg &middot; 18/07/2015 lúc 4:27 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    thank for share <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" /><br/>
    mặc dù đọc lần thứ n rồi vẫn thấy hay <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" />)))</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15141">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="hacker96pro" data-content="post-15142" id="js-post-15142">
    <span class="u-anchorTarget" id="post-15142"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/hacker96pro.41213/" class="avatar avatar--xs" data-user-id="41213" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/41/41213.jpg?1489802958" srcset="/data/avatars/m/41/41213.jpg?1489802958 2x" alt="hacker96pro" class="avatar-u41213-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/hacker96pro.41213/" class="username " dir="auto" itemprop="name" data-user-id="41213" data-xf-init="member-tooltip"><span class="username--style2">hacker96pro</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15142" data-lb-caption-desc="hacker96pro &middot; 18/07/2015 lúc 11:54 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    <blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch">
    <div class="bbCodeBlock-title">
    madawg;26790 đã viết:
    </div>
    <div class="bbCodeBlock-content">
    <div class="bbCodeBlock-expandContent js-expandContent ">
    thank for share <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" /><br/>
    mặc dù đọc lần thứ n rồi vẫn thấy hay <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" />)))
    </div>
    <div class="bbCodeBlock-expandLink js-expandLink"><a role="button" tabindex="0">Nhấn để mở rộng...</a></div>
    </div>
    </blockquote>Cái này hình như là của 1 blog nào đó <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite8" alt=":D" title="Big Grin    :D" loading="lazy" data-shortname=":D" /></div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15142">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="vigoss" data-content="post-15143" id="js-post-15143">
    <span class="u-anchorTarget" id="post-15143"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/vigoss.45210/" class="avatar avatar--xs avatar--default avatar--default--dynamic" data-user-id="45210" data-xf-init="member-tooltip" style="background-color: #cc9966; color: #604020">
    <span class="avatar-u45210-s">V</span>
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/vigoss.45210/" class="username " dir="auto" itemprop="name" data-user-id="45210" data-xf-init="member-tooltip"><span class="username--style2">vigoss</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15143" data-lb-caption-desc="vigoss &middot; 19/07/2015 lúc 3:07 AM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    <a href="http://www.procul.org/blog/2012/05/02/lam-an-toan-thong-tin-thi-h%E1%BB%8Dc-gi/comment-page-1/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.procul.org%2Fblog%2F2012%2F05%2F02%2Flam-an-toan-thong-tin-thi-h%25E1%25BB%258Dc-gi%2Fcomment-page-1%2F&amp;hash=0b307fe023129f233a49914595110e6b" rel="nofollow ugc noopener">http://www.procul.org/blog/2012/05/02/lam-an-toan-thong-tin-thi-học-gi/comment-page-1/</a> <br/>
    nguồn đây ! híc ! đọc tài liệu english nhức đầu kinh khủng ... với vốn từ vựng ít ỏi ! <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite3" alt=":(" title="Frown    :(" loading="lazy" data-shortname=":(" /></div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15143">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="whf" data-content="post-15144" id="js-post-15144">
    <span class="u-anchorTarget" id="post-15144"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/whf.22/" class="avatar avatar--xs" data-user-id="22" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/0/22.jpg?1500608303" srcset="/data/avatars/m/0/22.jpg?1500608303 2x" alt="whf" class="avatar-u22-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/whf.22/" class="username " dir="auto" itemprop="name" data-user-id="22" data-xf-init="member-tooltip"><span class="username--style439 username--staff">whf</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15144" data-lb-caption-desc="whf &middot; 19/07/2015 lúc 9:34 AM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    <blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch">
    <div class="bbCodeBlock-title">
    madawg;26790 đã viết:
    </div>
    <div class="bbCodeBlock-content">
    <div class="bbCodeBlock-expandContent js-expandContent ">
    thank for share <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" /><br/>
    mặc dù đọc lần thứ n rồi vẫn thấy hay <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" />)))
    </div>
    <div class="bbCodeBlock-expandLink js-expandLink"><a role="button" tabindex="0">Nhấn để mở rộng...</a></div>
    </div>
    </blockquote><br/>
    <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" /><br/>
    <br/>
    <blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch">
    <div class="bbCodeBlock-title">
    hacker96pro;26798 đã viết:
    </div>
    <div class="bbCodeBlock-content">
    <div class="bbCodeBlock-expandContent js-expandContent ">
    Cái này hình như là của 1 blog nào đó <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite8" alt=":D" title="Big Grin    :D" loading="lazy" data-shortname=":D" />
    </div>
    <div class="bbCodeBlock-expandLink js-expandLink"><a role="button" tabindex="0">Nhấn để mở rộng...</a></div>
    </div>
    </blockquote><br/>
    Đúng rồi bạn, đầu bài viết mình có giới thiệu về tác giả đó.<br/>
    <br/>
    <blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch">
    <div class="bbCodeBlock-title">
    vigoss;26799 đã viết:
    </div>
    <div class="bbCodeBlock-content">
    <div class="bbCodeBlock-expandContent js-expandContent ">
    <a href="http://www.procul.org/blog/2012/05/02/lam-an-toan-thong-tin-thi-h%E1%BB%8Dc-gi/comment-page-1/" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fwww.procul.org%2Fblog%2F2012%2F05%2F02%2Flam-an-toan-thong-tin-thi-h%25E1%25BB%258Dc-gi%2Fcomment-page-1%2F&amp;hash=0b307fe023129f233a49914595110e6b" rel="nofollow ugc noopener">http://www.procul.org/blog/2012/05/02/lam-an-toan-thong-tin-thi-h%E1%BB%8Dc-gi/comment-page-1/</a> <br/>
    nguồn đây ! híc ! đọc tài liệu english nhức đầu kinh khủng ... với vốn từ vựng ít ỏi ! <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite3" alt=":(" title="Frown    :(" loading="lazy" data-shortname=":(" />
    </div>
    <div class="bbCodeBlock-expandLink js-expandLink"><a role="button" tabindex="0">Nhấn để mở rộng...</a></div>
    </div>
    </blockquote><br/>
    Theo mình thì muốn tiến xa ở ngành an toàn thông tin nói riêng và công nghệ thông tin nói chung thì tiếng Anh là hết sức quan trọng, vì những tài liệu hay, mới nhất thường là tiếng Anh, do đó muốn tiến bộ chỉ có cách... học, học và học <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" /></div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15144">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="sunny" data-content="post-15145" id="js-post-15145">
    <span class="u-anchorTarget" id="post-15145"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/sunny.41623/" class="avatar avatar--xs" data-user-id="41623" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/41/41623.jpg?1490171876" srcset="/data/avatars/m/41/41623.jpg?1490171876 2x" alt="sunny" class="avatar-u41623-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/sunny.41623/" class="username " dir="auto" itemprop="name" data-user-id="41623" data-xf-init="member-tooltip">sunny</a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15145" data-lb-caption-desc="sunny &middot; 20/07/2015 lúc 12:10 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    Bài viết được tổng hợp lại và có link download rất chi tiết. <br/>
    Thanks!</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15145">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="whf" data-content="post-15146" id="js-post-15146">
    <span class="u-anchorTarget" id="post-15146"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/whf.22/" class="avatar avatar--xs" data-user-id="22" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/0/22.jpg?1500608303" srcset="/data/avatars/m/0/22.jpg?1500608303 2x" alt="whf" class="avatar-u22-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/whf.22/" class="username " dir="auto" itemprop="name" data-user-id="22" data-xf-init="member-tooltip"><span class="username--style439 username--staff">whf</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-15146" data-lb-caption-desc="whf &middot; 21/07/2015 lúc 8:06 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Re: Làm an toàn thông tin thì học gì?<br/>
    <br/>
    Mình vừa cập nhật thêm một số link tải trực tiếp cho anh em tiện download rồi nhé <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite9" alt=":eek:" title="Eek!    :eek:" loading="lazy" data-shortname=":eek:" /></div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList is-active ">
    <ul class="reactionSummary">
    <li><span class="reaction reaction--small reaction--1" data-reaction-id="1"><i aria-hidden="true"></i><img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="reaction-sprite js-reaction" alt="Thích" title="Thích" /></span></li>
    </ul>
    <span class="u-srOnly">Reactions:</span>
    <a class="reactionsBar-link" href="/posts/15146/reactions" data-xf-click="overlay" data-cache="false"><bdi>newbielearner123</bdi></a>
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post15146">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="tem9x" data-content="post-23195" id="js-post-23195">
    <span class="u-anchorTarget" id="post-23195"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/tem9x.45415/" class="avatar avatar--xs avatar--default avatar--default--dynamic" data-user-id="45415" data-xf-init="member-tooltip" style="background-color: #a88a8a; color: #493636">
    <span class="avatar-u45415-s">T</span>
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/tem9x.45415/" class="username " dir="auto" itemprop="name" data-user-id="45415" data-xf-init="member-tooltip"><span class="username--style2">tem9x</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-23195" data-lb-caption-desc="tem9x &middot; 17/04/2017 lúc 10:27 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Năm 2017 đọc lại bài này vẫn có rất nhiều thứ mình chưa làm được <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" /> Nếu ko cố gắng thì 2018 quay lại đọc bài này mình vẫn sẽ chưa làm được. Đang có tình trạng bị loạn kiến thức các bác ợ @@</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post23195">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="D4rkc0d3" data-content="post-23198" id="js-post-23198">
    <span class="u-anchorTarget" id="post-23198"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/d4rkc0d3.58928/" class="avatar avatar--xs" data-user-id="58928" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/58/58928.jpg?1499339692" srcset="/data/avatars/m/58/58928.jpg?1499339692 2x" alt="D4rkc0d3" class="avatar-u58928-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/d4rkc0d3.58928/" class="username " dir="auto" itemprop="name" data-user-id="58928" data-xf-init="member-tooltip"><span class="username--style2">D4rkc0d3</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-23198" data-lb-caption-desc="D4rkc0d3 &middot; 18/04/2017 lúc 2:46 AM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Có bạn nào muốn lập thành 1 nhóm học lớp CS140 ko <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="smilie smilie--sprite smilie--sprite1" alt=":)" title="Smile    :)" loading="lazy" data-shortname=":)" />)</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList is-active ">
    <ul class="reactionSummary">
    <li><span class="reaction reaction--small reaction--1" data-reaction-id="1"><i aria-hidden="true"></i><img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="reaction-sprite js-reaction" alt="Thích" title="Thích" /></span></li>
    </ul>
    <span class="u-srOnly">Reactions:</span>
    <a class="reactionsBar-link" href="/posts/23198/reactions" data-xf-click="overlay" data-cache="false"><bdi>whf</bdi></a>
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post23198">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="Tùng đẹp trai" data-content="post-34200" id="js-post-34200">
    <span class="u-anchorTarget" id="post-34200"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/tung-dep-trai.73498/" class="avatar avatar--xs avatar--default avatar--default--dynamic" data-user-id="73498" data-xf-init="member-tooltip" style="background-color: #e0e085; color: #8f8f24">
    <span class="avatar-u73498-s">T</span>
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/tung-dep-trai.73498/" class="username " dir="auto" itemprop="name" data-user-id="73498" data-xf-init="member-tooltip"><span class="username--style2">Tùng đẹp trai</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-34200" data-lb-caption-desc="Tùng đẹp trai &middot; 04/09/2019 lúc 8:25 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">Có ai có thể chi em biết trên bài có câu : học kiến thức căn bản thật vững (muốn biết thì coi phần 4) vậy em coi phần 4 ở đâu ạ mong mọi ng cho em biết</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post34200">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="whf" data-content="post-34204" id="js-post-34204">
    <span class="u-anchorTarget" id="post-34204"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/whf.22/" class="avatar avatar--xs" data-user-id="22" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/0/22.jpg?1500608303" srcset="/data/avatars/m/0/22.jpg?1500608303 2x" alt="whf" class="avatar-u22-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/whf.22/" class="username " dir="auto" itemprop="name" data-user-id="22" data-xf-init="member-tooltip"><span class="username--style439 username--staff">whf</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-34204" data-lb-caption-desc="whf &middot; 04/09/2019 lúc 10:11 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper"><blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch">
    <div class="bbCodeBlock-title">
    <a href="/goto/post?id=34200" class="bbCodeBlock-sourceJump" rel="nofollow" data-xf-click="attribution" data-content-selector="#post-34200">Tùng đẹp trai đã viết:</a>
    </div>
    <div class="bbCodeBlock-content">
    <div class="bbCodeBlock-expandContent js-expandContent ">
    Có ai có thể chi em biết trên bài có câu : học kiến thức căn bản thật vững (muốn biết thì coi phần 4) vậy em coi phần 4 ở đâu ạ mong mọi ng cho em biết
    </div>
    <div class="bbCodeBlock-expandLink js-expandLink"><a role="button" tabindex="0">Nhấn để mở rộng...</a></div>
    </div>
    </blockquote><br/>
    Bạn xem phần &quot;3. Học cái gì?&quot; nhé.</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList  uw_rec_logout">
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post34204">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="C4ACK34" data-content="post-36716" id="js-post-36716">
    <span class="u-anchorTarget" id="post-36716"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/c4ack34.78548/" class="avatar avatar--xs" data-user-id="78548" data-xf-init="member-tooltip">
    <img src="/data/avatars/s/78/78548.jpg?1588612142" srcset="/data/avatars/m/78/78548.jpg?1588612142 2x" alt="C4ACK34" class="avatar-u78548-s" width="48" height="48" loading="lazy" itemprop="image" />
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/c4ack34.78548/" class="username " dir="auto" itemprop="name" data-user-id="78548" data-xf-init="member-tooltip"><span class="username--style2">C4ACK34</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-36716" data-lb-caption-desc="C4ACK34 &middot; 29/05/2020 lúc 11:59 PM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">một số link sách bị die rồi anh ạ</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList is-active ">
    <ul class="reactionSummary">
    <li><span class="reaction reaction--small reaction--1" data-reaction-id="1"><i aria-hidden="true"></i><img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="reaction-sprite js-reaction" alt="Thích" title="Thích" /></span></li>
    </ul>
    <span class="u-srOnly">Reactions:</span>
    <a class="reactionsBar-link" href="/posts/36716/reactions" data-xf-click="overlay" data-cache="false"><bdi>whf</bdi></a>
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post36716">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    <article class="message message--post js-post js-inlineModContainer   binhluan" data-author="KUNGFU" data-content="post-36938" id="js-post-36938">
    <span class="u-anchorTarget" id="post-36938"></span>
    <div class="message-inner">
    <div class="message-cell message-cell--user">
    </div>
    <div class="message-cell message-cell--main">
    <div class="message-main js-quickEditTarget">
    <header class="message-attribution">
    <div class="message-attribution-main listInline ">
    <a href="/members/kungfu.79100/" class="avatar avatar--xs avatar--default avatar--default--dynamic" data-user-id="79100" data-xf-init="member-tooltip" style="background-color: #66cc33; color: #1f3d0f">
    <span class="avatar-u79100-s">K</span>
    </a>
    <div class="info-message">
    <div class="button-follow follow-cmt">
    </div>
    <ul class="info">
    <li class="u-concealed">
    <a href="/members/kungfu.79100/" class="username " dir="auto" itemprop="name" data-user-id="79100" data-xf-init="member-tooltip"><span class="username--style2">KUNGFU</span></a>
    </li>
    </ul>
    </div>
    </div>
    </header>
    <div class="message-content js-messageContent">
    <div class="message-userContent lbContainer js-lbContainer " data-lb-id="post-36938" data-lb-caption-desc="KUNGFU &middot; 17/06/2020 lúc 2:01 AM">
    <article class="message-body js-selectToQuote">
    <div class="bbWrapper">tải sao em mở mấy đường link lập trình c không được ạ,ai hổ trở em với.<br/>
    <a href="http://jarson.asia/resource/CSAPP.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fjarson.asia%2Fresource%2FCSAPP.pdf&amp;hash=161edacd7a04aa3d0e2eb6c0b14f0e68" rel="nofollow ugc noopener">Randal Bryant, David O'Hallaron, Computer Systems: A Programmer's Perspective</a>:<br/>
    <a href="https://hassanolity.files.wordpress.com/2013/11/the_c_programming_language_2.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=https%3A%2F%2Fhassanolity.files.wordpress.com%2F2013%2F11%2Fthe_c_programming_language_2.pdf&amp;hash=410b313450755a5db14a9ff59617090a" rel="nofollow ugc noopener">Brian Kernighan, Dennis Ritchie, The C Programming Language (2nd Edition)</a>:<br/>
    <a href="http://forge.huihoo.com/attachments/download/682/c-interfaces-and-implementations-techniques-for-creating-reusable-software.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Fforge.huihoo.com%2Fattachments%2Fdownload%2F682%2Fc-interfaces-and-implementations-techniques-for-creating-reusable-software.pdf&amp;hash=d4bc6e060a6ffb5bf9dcea2b689c1ef2" rel="nofollow ugc noopener">David Hanson, C Interfaces and Implementations</a>:<br/>
    <a href="http://files.stuxnode.com/Gray-Hat-Python.pdf" target="_blank" class="link link--external" data-proxy-href="/proxy.php?link=http%3A%2F%2Ffiles.stuxnode.com%2FGray-Hat-Python.pdf&amp;hash=244499b82fc98e7ac0be3d34dfb98e1a" rel="nofollow ugc noopener">Justin Seitz, Gray Hat Python</a>: <br/>
    Douglas Crockford, JavaScript: The Good Parts:</div>
    <div class="js-selectToQuoteEnd">&nbsp;</div>
    </article>
    </div>
    </div>
    <footer class="message-footer">
    <div style="    border-bottom: 1px dashed #a5cae4;
        margin-bottom: 10px;
        padding-bottom: 10px;
        border-top: 1px dashed #a5cae4;
        margin-top: 5px;
        padding-top: 10px;
        overflow: auto; width: 100%;"><span class="js-one" style="float: right;"><i>Mời các bạn tham gia <a href="https://www.facebook.com/groups/2217764565218392">Group WhiteHat</a> để thảo luận và cập nhật tin tức an ninh mạng hàng ngày.</i><br><b>Lưu ý từ WhiteHat:</b> Kiến thức an ninh mạng để phòng chống, không làm điều xấu. <a href="https://whitehat.vn/threads/mot-so-luat-hinh-su-lien-quan-den-an-ninh-an-toan-thong-tin.330/">Luật pháp liên quan</a></span></div>
    <div class="reactionsBar js-reactionsList is-active ">
    <ul class="reactionSummary">
    <li><span class="reaction reaction--small reaction--1" data-reaction-id="1"><i aria-hidden="true"></i><img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" class="reaction-sprite js-reaction" alt="Thích" title="Thích" /></span></li>
    </ul>
    <span class="u-srOnly">Reactions:</span>
    <a class="reactionsBar-link" href="/posts/36938/reactions" data-xf-click="overlay" data-cache="false"><bdi>whf</bdi></a>
    </div>
    <div class="js-historyTarget message-historyTarget toggleTarget" data-href="trigger-href"></div>
    </footer>
    <section class="message-responses js-messageResponses uw_fcs_comment_section">
    <div class="message-responseRow u-jsOnly js-commentLoader p-description uw-comment-count uw-hidden ">
    <i class="far fa-comment"></i> <span class="comment-count" data-count="0"></span>
    Comment
    </div>
    <div class="js-replyNewMessageContainer-post36938">
    </div>
    </section>
    </div>
    </div>
    </div>
    </article>
    </div>
    </div>
    <div class="block-outer block-outer--after">
    <div class="block-outer-main"><nav class="pageNavWrapper pageNavWrapper--full ">
    <div class="pageNav  ">
    <ul class="pageNav-main">
    <li class="pageNav-page pageNav-page--current "><a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/">1</a></li>
    <li class="pageNav-page "><a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2">2</a></li>
    </ul>
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2" class="pageNav-jump pageNav-jump--next">Tiếp</a>
    </div>
    <div class="pageNavSimple">
    <a class="pageNavSimple-el pageNavSimple-el--current" data-xf-init="tooltip" title="Đi tới trang" data-xf-click="menu" role="button" tabindex="0" aria-expanded="false" aria-haspopup="true">
    1 of 2
    </a>
    <div class="menu menu--pageJump" data-menu="menu" aria-hidden="true">
    <div class="menu-content">
    <h4 class="menu-header">Đi tới trang</h4>
    <div class="menu-row" data-xf-init="page-jump" data-page-url="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-%page%">
    <div class="inputGroup inputGroup--numbers">
    <div class="inputGroup inputGroup--numbers inputNumber" data-xf-init="number-box"><input type="number" pattern="\d*" class="input input--number js-numberBoxTextInput input input--numberNarrow js-pageJumpPage" value="1" min="1" max="2" step="1" required="required" data-menu-autofocus="true" /></div>
    <span class="inputGroup-text"><button type="button" class="js-pageJumpGo button"><span class="button-text">Tới</span></button></span>
    </div>
    </div>
    </div>
    </div>
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2" class="pageNavSimple-el pageNavSimple-el--next">
    Tiếp <i aria-hidden="true"></i>
    </a>
    <a href="/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/page-2" class="pageNavSimple-el pageNavSimple-el--last" data-xf-init="tooltip" title="Cuối">
    <i aria-hidden="true"></i> <span class="u-srOnly">Cuối</span>
    </a>
    </div>
    </nav>
    </div>
    <div class="block-outer-opposite">
    <a href="/login/" class="button--link button--wrap button" data-xf-click="overlay"><span class="button-text">
    Bạn phải đăng nhập hoặc đăng ký để phản hồi tại đây.
    </span></a>
    </div>
    </div>
    <div class="block block-moreThread block-default block-relatedThreads">
    <div class="block-container">
    <div class="block-minorHeader">
    Bài viết liên quan
    </div>
    <div class="block-body">
    <li class="block-row">
    <div class="contentRow">
    <a class="thumbnail avatar" href="/threads/xin-tai-lieu-ve-an-toan-thong-tin-ccsp.18045/" style="background-image: url('https://i.imgur.com/Ls4jCmF.jpg')">
    </a>
    <div class="contentRow-main">
    <a href="/threads/xin-tai-lieu-ve-an-toan-thong-tin-ccsp.18045/" title="Xin tài liệu về An toàn thông tin CCSP" class="title" data-xf-init="tooltip">
    Xin tài liệu về An toàn thông tin CCSP
    </a>
    <span class="desc">
    <ul class="listInline listInline--bullet">
    <li>
    <i class="fa--xf far fa-clock" aria-hidden="true" title="Ngày bắt đầu"></i>
    <span class="u-srOnly">Ngày bắt đầu</span>
    <a href="/threads/xin-tai-lieu-ve-an-toan-thong-tin-ccsp.18045/" class="u-concealed"><time class="u-dt" dir="auto" datetime="2024-07-22T10:55:27+0700" data-time="1721620527" data-date-string="22/07/2024" data-time-string="10:55 AM" title="22/07/2024 lúc 10:55 AM">22/07/2024</time></a>
    </li>
    <li>
    <i class="fa--xf far fa-comment" aria-hidden="true"></i>
    0
    </li>
    </ul>
    </span>
    </div>
    </div>
    </li>
    <li class="block-row">
    <div class="contentRow">
    <a class="thumbnail avatar" href="/threads/xin-tai-lieu-cho-nguoi-moi-bat-dau-ve-mang-may-tinh.17106/" style="background-image: url('https://i.imgur.com/Ls4jCmF.jpg')">
    </a>
    <div class="contentRow-main">
    <a href="/threads/xin-tai-lieu-cho-nguoi-moi-bat-dau-ve-mang-may-tinh.17106/" title="Xin tài liệu cho người mới bắt đầu về mạng máy tính" class="title" data-xf-init="tooltip">
    Xin tài liệu cho người mới bắt đầu về mạng máy tính
    </a>
    <span class="desc">
    <ul class="listInline listInline--bullet">
    <li>
    <i class="fa--xf far fa-clock" aria-hidden="true" title="Ngày bắt đầu"></i>
    <span class="u-srOnly">Ngày bắt đầu</span>
    <a href="/threads/xin-tai-lieu-cho-nguoi-moi-bat-dau-ve-mang-may-tinh.17106/" class="u-concealed"><time class="u-dt" dir="auto" datetime="2023-01-31T19:06:19+0700" data-time="1675166779" data-date-string="31/01/2023" data-time-string="7:06 PM" title="31/01/2023 lúc 7:06 PM">31/01/2023</time></a>
    </li>
    <li>
    <i class="fa--xf far fa-comment" aria-hidden="true"></i>
    2
    </li>
    </ul>
    </span>
    </div>
    </div>
    </li>
    <li class="block-row">
    <div class="contentRow">
    <a class="thumbnail avatar" href="/threads/lam-the-nao-de-khai-thac-lo-hong-zeroday-0-day.8767/" style="background-image: url('https://i.imgur.com/Ls4jCmF.jpg')">
    </a>
    <div class="contentRow-main">
    <a href="/threads/lam-the-nao-de-khai-thac-lo-hong-zeroday-0-day.8767/" title="Làm thế nào để khai thác lỗ hổng zeroday (0 day) ??" class="title" data-xf-init="tooltip">
    Làm thế nào để khai thác lỗ hổng zeroday (0 day) ??
    </a>
    <span class="desc">
    <ul class="listInline listInline--bullet">
    <li>
    <i class="fa--xf far fa-clock" aria-hidden="true" title="Ngày bắt đầu"></i>
    <span class="u-srOnly">Ngày bắt đầu</span>
    <a href="/threads/lam-the-nao-de-khai-thac-lo-hong-zeroday-0-day.8767/" class="u-concealed"><time class="u-dt" dir="auto" datetime="2017-05-06T12:07:07+0700" data-time="1494047227" data-date-string="06/05/2017" data-time-string="12:07 PM" title="06/05/2017 lúc 12:07 PM">06/05/2017</time></a>
    </li>
    <li>
    <i class="fa--xf far fa-comment" aria-hidden="true"></i>
    2
    </li>
    </ul>
    </span>
    </div>
    </div>
    </li>
    <li class="block-row">
    <div class="contentRow">
    <a class="thumbnail avatar" href="/threads/tron-bo-tai-lieu-hoc-lam-hacker-tu-co-ban-den-nang-cao.8701/" style="background-image: url('https://4.bp.blogspot.com/-n0mtE6t8OJI/V56lKGm4xFI/AAAAAAAAQJA/7RbTIIzyNnwVPZXxwZkbmSeX1zgraWQPgCLcB/s1600/2.png')">
    </a>
    <div class="contentRow-main">
    <a href="/threads/tron-bo-tai-lieu-hoc-lam-hacker-tu-co-ban-den-nang-cao.8701/" title="Trọn bộ tài liệu học làm hacker từ cơ bản đến nâng cao" class="title" data-xf-init="tooltip">
    Trọn bộ tài liệu học làm hacker từ cơ bản đến nâng cao
    </a>
    <span class="desc">
    <ul class="listInline listInline--bullet">
    <li>
    <i class="fa--xf far fa-clock" aria-hidden="true" title="Ngày bắt đầu"></i>
    <span class="u-srOnly">Ngày bắt đầu</span>
    <a href="/threads/tron-bo-tai-lieu-hoc-lam-hacker-tu-co-ban-den-nang-cao.8701/" class="u-concealed"><time class="u-dt" dir="auto" datetime="2017-04-26T17:31:08+0700" data-time="1493202668" data-date-string="26/04/2017" data-time-string="5:31 PM" title="26/04/2017 lúc 5:31 PM">26/04/2017</time></a>
    </li>
    <li>
    <i class="fa--xf far fa-comment" aria-hidden="true"></i>
    20
    </li>
    </ul>
    </span>
    </div>
    </div>
    </li>
    <li class="block-row">
    <div class="contentRow">
    <a class="thumbnail avatar" href="/threads/huong-dan-su-dung-hyper-v-lam-test-lab-tren-ws-2012r2.1201/" style="background-image: url('http://i.imgur.com/dnsOpxv.png')">
    </a>
    <div class="contentRow-main">
    <a href="/threads/huong-dan-su-dung-hyper-v-lam-test-lab-tren-ws-2012r2.1201/" title="Hướng dẫn sử dụng Hyper-v làm test lab trên ws 2012R2" class="title" data-xf-init="tooltip">
    Hướng dẫn sử dụng Hyper-v làm test lab trên ws 2012R2
    </a>
    <span class="desc">
    <ul class="listInline listInline--bullet">
    <li>
    <i class="fa--xf far fa-clock" aria-hidden="true" title="Ngày bắt đầu"></i>
    <span class="u-srOnly">Ngày bắt đầu</span>
    <a href="/threads/huong-dan-su-dung-hyper-v-lam-test-lab-tren-ws-2012r2.1201/" class="u-concealed"><time class="u-dt" dir="auto" datetime="2014-04-30T00:39:57+0700" data-time="1398793197" data-date-string="30/04/2014" data-time-string="12:39 AM" title="30/04/2014 lúc 12:39 AM">30/04/2014</time></a>
    </li>
    <li>
    <i class="fa--xf far fa-comment" aria-hidden="true"></i>
    4
    </li>
    </ul>
    </span>
    </div>
    </div>
    </li>
    <li class="block-row">
    <div class="contentRow">
    <a class="thumbnail avatar" href="/threads/secure-sockets-layer-ssl-va-cach-lam-viec.497/" style="background-image: url('https://whitehat.vn/image/xenforo_image/1490892964http1.jpg')">
    </a>
    <div class="contentRow-main">
    <a href="/threads/secure-sockets-layer-ssl-va-cach-lam-viec.497/" title="Secure Sockets Layer (SSL) và cách làm việc" class="title" data-xf-init="tooltip">
    Secure Sockets Layer (SSL) và cách làm việc
    </a>
    <span class="desc">
    <ul class="listInline listInline--bullet">
    <li>
    <i class="fa--xf far fa-clock" aria-hidden="true" title="Ngày bắt đầu"></i>
    <span class="u-srOnly">Ngày bắt đầu</span>
    <a href="/threads/secure-sockets-layer-ssl-va-cach-lam-viec.497/" class="u-concealed"><time class="u-dt" dir="auto" datetime="2013-12-11T15:21:28+0700" data-time="1386750088" data-date-string="11/12/2013" data-time-string="3:21 PM" title="11/12/2013 lúc 3:21 PM">11/12/2013</time></a>
    </li>
    <li>
    <i class="fa--xf far fa-comment" aria-hidden="true"></i>
    2
    </li>
    </ul>
    </span>
    </div>
    </div>
    </li>
    </div>
    </div>
    </div>
    </div>
    </div>
    </div>
    </div>
    </div>
    </div>
    <footer class="p-footer" id="footer">
    <div class="p-footer-inner">
    <div class="Fbottom1">
    <div class="p-footer-inner Flist1">
    <div class="cl-1">
    <img src="/styles/whitehat-2021/logo-n.png" alt="logo footer">
    <p>@ 2009 - <script type="ffcfcb791e2c54b60847998b-text/javascript">document.write(new Date().getFullYear())</script> Bkav Cyber Security</p>
    </div>
    <div class="cl-1">
    <h2>Chuyên mục</h2>
    <p class="text-muted"></p>
    <li class="media"><a target="_blank" href="/tin-tuc-an-ninh-mang/">Tin tức</a></li>
    <li class="media"><a target="_blank" href="https://wargame.whitehat.vn/">WarGame</a></li>
    <li class="media"><a target="_blank" href="/forums/">Thảo luận</a></li>
    <li class="media"><a target="_blank" href="/video/">Video</a></li>
    </div>
    <div class="cl-1 middle">
    <div class="dropdown">
    <button value="laptrinh" class="dropbtn">Số người đang xem</button>
    <div id="laptrinh" class="dropdown-content">
    <div class="block" data-widget-id="13" data-widget-key="xc_user_activity_view_for" data-widget-definition="xc_user_activity_view_for">
    <div class="block-container block-userActivity">
    <h3 class="block-header block-userActivity--header">
    Số người đang xem
    </h3>
    <div class="block-body">
    <a href="/online/">
    <p style="font-size: 11px; color: white; font-weight: normal; margin: 0;">
    Hiện đang có 0 người trực tuyến: 0 Thành viên 0 Khách
    </p>
    </a>
    <p style="margin: 0;" id="number-of-access-user">Nhiều người truy cập nhất: 10.746, 15/07/2014 vào lúc 04:04 PM.</p>
    </div>
    </div>
    </div>
    </div>
    </div>
    <div class="dropdown">
    <button value="thuthuat" class="dropbtn">Thống kê - WhiteHat Forum</button>
    <div id="thuthuat" class="dropdown-content">
    <div class="block" data-widget-id="7" data-widget-key="forum_overview_forum_statistics" data-widget-definition="forum_statistics">
    <div class="block-container">
    <h3 class="block-minorHeader">Thống kê diễn đàn</h3>
    <div class="block-body block-row">
    <div class="h-margin-bottom-s mobile-hide" style="color:white; font-size: 11px;">
    Chủ đề: <span id="count_thread">11.989</span> &nbsp;
    Bài viết: <span id="count_post">33.728</span> &nbsp;
    Thành viên: <span id="count_user">90.601</span> &nbsp;
    </div>
    <div style="color:white; font-size: 11px;" class="mobile-hide">
    Chào mừng thành viên mới nhất của diễn đàn, <a href="/members/66bet88com.114441/" class="username " dir="auto" itemprop="name" data-user-id="114441" data-xf-init="member-tooltip">66bet88com</a>
    </div>
    </div>
    </div>
    </div>
    </div>
    </div>
    </div>
    <div class="cl-1 last">
    <li class="media">Tòa nhà HH1, Khu đô thị Yên Hòa, Cầu Giấy, Hà Nội</li>
    <li class="media">Giấy phép MXH số 355/GP - BTTT do BTTTT cấp</li>
    <li class="media">Ghi rõ 'nguồn Bkav Cyber Security' khi phát hành lại thông tin từ Website này</li>
    </div>
    </div>
    </div>
    </div>
    </footer>
    </div> 
    <div class="u-bottomFixer js-bottomFixTarget">
    </div>
    <div class="u-scrollButtons js-scrollButtons" data-trigger-type="up">
    <a href="#top" class="button--scroll button" data-xf-click="scroll-to"><span class="button-text"><i class="fa--xf far fa-arrow-up" aria-hidden="true"></i><span class="u-srOnly">Bên trên</span></span></a>
    </div>
    <script src="/js/vendor/jquery/jquery-3.5.1.min.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/vendor/vendor-compiled.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/xf/core-compiled.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/xf/message.min.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/UW/FCS/editor.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/UW/FCS/comment-main.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/xf/comment.min.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script src="/js/DBTech/UserTagging/core.min.js?_v=60f41f6b" type="ffcfcb791e2c54b60847998b-text/javascript"></script>
    <script type="ffcfcb791e2c54b60847998b-text/javascript">

        var header = document.querySelector('.message-user');
        var origOffsetY = header.offsetTop;

        function onScroll(e) {
            window.scrollY - 300 >= origOffsetY ? header.classList.add('sticky') :
            header.classList.remove('sticky');
        }

        document.addEventListener('scroll', onScroll);


    </script>
    <script type="ffcfcb791e2c54b60847998b-text/javascript">

    var randomNum = '?v=' + Math.ceil(Math.random() * 999999);
        $('img').each(function(){
            if($(this).attr('src').indexOf('data/articles') > -1) {
                var lin = $(this).attr('src')+randomNum;
                $(this).attr('src', lin)
            }
        });
        $('.thumb > div').each(function(){
            if($(this).attr('style').indexOf('data/articles') > -1) {
                var lin = $(this).attr('style').replace('.jpg', '.jpg'+randomNum);
                $(this).attr('style', lin)
            }
        });
        /* Thêm hoặc xóa class show ra khỏi phần tử */
        function myFunction(id) {
            document.getElementById(id).classList.toggle("show");
        }
        //lấy tất cả các button menu
        var buttons = document.getElementsByClassName('dropbtn');
        //lấy tất cả các thẻ chứa menu con
        var contents = document.getElementsByClassName('dropdown-content');
        //lặp qua tất cả các button menu và gán sự kiện
        for (var i = 0; i < buttons.length; i++) {
            buttons[i].addEventListener("click", function(){
                //lấy value của button
                var id = this.value;
                //ẩn tất cả các menu con đang được hiển thị
                for (var i = 0; i < contents.length; i++) {
                    contents[i].classList.remove("show");
                }
                //hiển thị menu vừa được click
                myFunction(id);
            });
        }
        //nếu click ra ngoài các button thì ẩn tất cả các menu con
        window.addEventListener("click", function(){
            if (!event.target.matches('.dropbtn')){
                for (var i = 0; i < contents.length; i++) {
                    contents[i].classList.remove("show");
                }
            }
        });

    </script>
    <script type="ffcfcb791e2c54b60847998b-text/javascript">

        $('.p-breadcrumbs > li > a[href="/categories/wargame.45/"]').attr('href', '/wargame/')

    </script>
    <script type="ffcfcb791e2c54b60847998b-text/javascript">
        const CMTV_MATH_RENDER_OPTIONS =
        {
            "delimiters":
            [
                {left: "[imath]", right: "[/imath]", display: false},
                {left: "[math]",  right: "[/math]", display: true}
            ].concat([
        { "left": "$$", "right": "$$", "display": true },
        { "left": "$",  "right": "$",  "display": false },
        
        { "left": "\\[", "right": "\\]", "display": true },
        { "left": "\\(", "right": "\\)", "display": false }
    ]),

            "ignoredClasses": [].concat([
        "fr-wrapper", 	// Froala editor
        "username"		// Username
    ]),

            "macros": {
        // Parameterless example (short version to write limit for sequences, where n goes to infinity)
        //"\\limn":   "\\lim\\limits_{n\\to\\infty}",
        
        // Macros with one parameter example (floor of the number)
        //"\\floor":  "\\left\\lfloor #1 \\right\\rfloor",
    }
        };
        
        (function ($, document)
        {
            $(document).on('xf:reinit', function (e)
            {
                renderMathInElement(document.body, CMTV_MATH_RENDER_OPTIONS);
            });
        })
        (jQuery, document);
    </script>
    <script type="ffcfcb791e2c54b60847998b-text/javascript">
            jQuery.extend(true, XF.config, {
                // 
                userId: 0,
                enablePush: false,
                pushAppServerKey: '',
                url: {
                    fullBase: 'https://whitehat.vn/',
                    basePath: '/',
                    css: '/css.php?css=__SENTINEL__&s=5&l=1&d=1710560919',
                    keepAlive: '/login/keep-alive'
                },
                cookie: {
                    path: '/',
                    domain: '',
                    prefix: 'xf_',
                    secure: true
                },
                cacheKey: 'b902a0efa87bfe0e657c238d1243962a',
                csrf: '1723178075,3a6deb8fb7fcf51c78bb639afad7a146',
                js: {"\/js\/xf\/message.min.js?_v=60f41f6b":true,"\/js\/UW\/FCS\/editor.js?_v=60f41f6b":true,"\/js\/UW\/FCS\/comment-main.js?_v=60f41f6b":true,"\/js\/xf\/comment.min.js?_v=60f41f6b":true,"\/js\/DBTech\/UserTagging\/core.min.js?_v=60f41f6b":true},
                css: {"public:CMTV_Math.less":true,"public:bb_code.less":true,"public:dcMoreThread.less":true,"public:footer_custom.less":true,"public:message.less":true,"public:nulumia_breadcrumb_essentials.less":true,"public:share_controls.less":true,"public:xc_user_activity.less":true,"public:extra.less":true},
                time: {
                    now: 1723178075,
                    today: 1723136400,
                    todayDow: 5,
                    tomorrow: 1723222800,
                    yesterday: 1723050000,
                    week: 1722618000
                },
                borderSizeFeature: '3px',
                fontAwesomeWeight: 'r',
                enableRtnProtect: true,
                enableFormSubmitSticky: true,
                uploadMaxFilesize: 2097152,
                allowedVideoExtensions: ["m4v","mov","mp4","mp4v","mpeg","mpg","ogv","webm"],
                allowedAudioExtensions: ["mp3","ogg","wav"],
                shortcodeToEmoji: true,
                visitorCounts: {
                    conversations_unread: '0',
                    alerts_unviewed: '0',
                    total_unread: '0',
                    title_count: true,
                    icon_indicator: true
                },
                jsState: {},
                publicMetadataLogoUrl: 'https://whitehat.vn/data/assets/logo/wh-logo.png',
                publicPushBadgeUrl: 'https://whitehat.vn/styles/default/xenforo/bell.png'
            });

            jQuery.extend(XF.phrases, {
                // 
                date_x_at_time_y: "{date} lúc {time}",
                day_x_at_time_y:  "{day} lúc {time}",
                yesterday_at_x:   "Hôm qua, lúc {time}",
                x_minutes_ago:    "{minutes} phút trước",
                one_minute_ago:   "1 phút trước",
                a_moment_ago:     "Vài giây trước",
                today_at_x:       "Hôm nay lúc {time}",
                in_a_moment:      "Trong chốc lát",
                in_a_minute:      "In a minute",
                in_x_minutes:     "In {minutes} minutes",
                later_today_at_x: "Cuối ngày hôm nay lúc {time}",
                tomorrow_at_x:    "Ngày mai lúc {time}",

                day0: "Chủ nhật",
                day1: "Thứ hai",
                day2: "Thứ ba",
                day3: "Thứ tư",
                day4: "Thứ năm",
                day5: "Thứ sáu",
                day6: "Thứ bảy",

                dayShort0: "CN",
                dayShort1: "T2",
                dayShort2: "T3",
                dayShort3: "T4",
                dayShort4: "T5",
                dayShort5: "T6",
                dayShort6: "T7",

                month0: "Tháng một",
                month1: "Tháng hai",
                month2: "Tháng ba",
                month3: "Tháng tư",
                month4: "Tháng năm",
                month5: "Tháng sáu",
                month6: "Tháng bảy",
                month7: "Tháng tám",
                month8: "Tháng chín",
                month9: "Tháng mười",
                month10: "Tháng mười một",
                month11: "Tháng mười hai",

                active_user_changed_reload_page: "The active user has changed. Reload the page for the latest version.",
                server_did_not_respond_in_time_try_again: "The server did not respond in time. Please try again.",
                oops_we_ran_into_some_problems: "OMG! Có rắc rối rồi.",
                oops_we_ran_into_some_problems_more_details_console: "OMG! Có rắc rối rồi. Vui lòng thử lại. Chi tiết lỗi có thể hiển thị trong bảng điều khiển của trình duyệt.",
                file_too_large_to_upload: "The file is too large to be uploaded.",
                uploaded_file_is_too_large_for_server_to_process: "The uploaded file is too large for the server to process.",
                files_being_uploaded_are_you_sure: "Files are still being uploaded. Are you sure you want to submit this form?",
                attach: "Attach files",
                rich_text_box: "Rich text box",
                close: "Đóng",
                link_copied_to_clipboard: "Liên kết đã được sao chép.",
                text_copied_to_clipboard: "Text copied to clipboard.",
                loading: "Đang tải…",

                processing: "Processing",
                'processing...': "Processing…",

                showing_x_of_y_items: "Showing {count} of {total} items",
                showing_all_items: "Showing all items",
                no_items_to_display: "Không có mục nào để hiển thị",

                number_button_up: "Increase",
                number_button_down: "Decrease",

                push_enable_notification_title: "Chức năng gửi thông báo đã được bật trên trang WhiteHat.vn",
                push_enable_notification_body: "Cám ơn bạn đã đăng ký nhận thông báo!"
            ,
                "svStandardLib_time.day": "{count} day",
                "svStandardLib_time.days": "{count} days",
                "svStandardLib_time.hour": "{count} hour",
                "svStandardLib_time.hours": "{count} hours",
                "svStandardLib_time.minute": "{count} minutes",
                "svStandardLib_time.minutes": "{count} minutes",
                "svStandardLib_time.month": "{count} month",
                "svStandardLib_time.months": "{count} months",
                "svStandardLib_time.second": "{count} second",
                "svStandardLib_time.seconds": "{count} seconds",
                "svStandardLib_time.week": "time.week",
                "svStandardLib_time.weeks": "{count} weeks",
                "svStandardLib_time.year": "{count} year",
                "svStandardLib_time.years": "{count} years"

            });
        </script>
    <form style="display:none" hidden="hidden">
    <input type="text" name="_xfClientLoadTime" value id="_xfClientLoadTime" title="_xfClientLoadTime" tabindex="-1" />
    </form>
    <script type="application/ld+json">
                    {
        "@context": "https://schema.org",
        "@type": "DiscussionForumPosting",
        "@id": "https://whitehat.vn/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/",
        "headline": "L\u00e0m an to\u00e0n th\u00f4ng tin th\u00ec h\u1ecdc g\u00ec?",
        "articleBody": "An to\u00e0n th\u00f4ng tin l\u00e0 l\u00e0m g\u00ec, n\u00ean h\u1ecdc nh\u1eefng g\u00ec? \u0110\u00f3 l\u00e0 nh\u1eefng c\u00e2u h\u1ecfi m\u00e0 nhi\u1ec1u b\u1ea1n d\u1ef1 \u0111\u1ecbnh theo l\u0129nh v\u1ef1c an to\u00e0n th\u00f4ng tin c\u00f2n b\u0103n kho\u0103n. Do \u0111\u00f3 trong b\u00e0i n\u00e0y m\u00ecnh xin chia s\u1ebb b\u00e0i vi\u1ebft \"L\u00e0m an to\u00e0n th\u00f4ng tin th\u00ec h\u1ecdc g\u00ec?\" c\u1ee7a t\u00e1c gi\u1ea3 D\u01b0\u01a1ng Ng\u1ecdc Th\u00e1i -...",
        "articleSection": "T\u00e0i li\u1ec7u",
        "author": {
            "@type": "Person",
            "name": "whf"
        },
        "datePublished": "2015-06-28T16:55:11+00:00",
        "dateModified": "2023-05-12T02:01:32+00:00",
        "image": "https://whitehat.vn/data/avatars/l/0/22.jpg?1500608303",
        "interactionStatistic": {
            "@type": "InteractionCounter",
            "interactionType": "https://schema.org/ReplyAction",
            "userInteractionCount": 23
        },
        "publisher": {
            "@type": "Organization",
            "name": "WhiteHat.vn",
            "logo": {
                "@type": "ImageObject",
                "url": "https://whitehat.vn/data/assets/logo/wh-logo.png"
            }
        },
        "mainEntityOfPage": {
            "@type": "WebPage",
            "@id": "https://whitehat.vn/threads/lam-an-toan-thong-tin-thi-hoc-gi.5281/"
        }
    }
                </script>
    <script src="/cdn-cgi/scripts/7d0fa10a/cloudflare-static/rocket-loader.min.js" data-cf-settings="ffcfcb791e2c54b60847998b-|49" defer></script></body>
    </html>
    """

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
qlora_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_quant_storage=torch.float16,
    bnb_4bit_use_double_quant=True
) # `qlora_config` or `quantization_config` or `bitsandbytes_config` 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, 
    low_cpu_mem_usage=True,
    cache_dir="../../cache",
    use_cache=True, 
    quantization_config=qlora_config, 
    torch_dtype=torch.float16
)
# nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
class CustomDatasetWebCrawler(Dataset):
    """Custom Dataset for Web Crawler dataset."""
    def __init__(self, encodings):
        """
        Arguments:
            encodings: List(Dict{"hello": 1, "goodbye": 2, ...}, Dict{"gino": 56, "bravo": 80, ...}, ...)
        """
        super().__init__()
        self.encodings = encodings  

    def __len__(self):
        return len(self.encodings)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings["input_ids"][idx],
            'attention_mask': self.encodings["attention_mask"][idx],
        }

def format_prompts_func(dataframe, system_prompt=SYSTEM_PROMPT, user_prompt=USER_PROMPT, assistant_prompt=ASSISTANT_PROMPT):
    """
    Prepare data following the Llama3.1-instruct format0
    """
    html_sources = dataframe["HTML source"]
    urls = dataframe["URL"]
    headlines = dataframe["Headline"]
    opening_texts = dataframe["Opening Text"]
    hit_sentences = dataframe["Hit Sentence"]
    sources = dataframe["Source"]
    influencers = dataframe["Influencer"]
    countries = dataframe["Country"]
    languages = dataframe["Language"]
    texts = []
    for url, html_source, headline, opening_text, hit_sentence, source, influencer, country, language in zip(urls, html_sources, headlines, opening_texts, hit_sentences, sources, influencers, countries, languages):
        input = html_source
        output = {"URL": url,"Headline": headline, "Opening Text": opening_text, "Hit Sentence": hit_sentence, "Source": source,"Influencer": influencer, "Country": country,"Language": language}
        text = system_prompt + "\n\n\n" + user_prompt.format(input) + "\n\n\n" + assistant_prompt.format(output)
        texts.append(text)
    return {"texts" : texts}

def setup_dataset(dataset_name=DATASET_NAME):
    """
    Args:
        dataset_name: str
    Returns:
        Dict: {"key": List(str)}
    """
    dataset = load_dataset(dataset_name, split="train")
    dataset = dataset.map(format_prompts_func, batched=True)
    return dataset

In [13]:
dataset = setup_dataset(DATASET_NAME)

In [14]:
train_split = int(len(dataset["texts"]) * 0.8)

In [15]:
train_dataset = dataset["texts"][:train_split]
val_dataset = dataset["texts"][train_split:]

In [16]:
print(">>>>>>>>>>> Tokenizing dataset... <<<<<<<<<<<")
# train_dataset_encodings = [tokenizer(train_data, truncation=True, padding=True, max_length=130_000, return_tensors="pt") for train_data in tqdm.tqdm(train_dataset)]
# val_dataset_encodings = [tokenizer(valid_data, truncation=True, padding=True, max_length=130_000, return_tensors="pt") for valid_data in tqdm.tqdm(val_dataset)]

train_dataset_encodings = tokenizer(train_dataset, truncation=True, padding=True, max_length=130_000, return_tensors="pt")
val_dataset_encodings = tokenizer(val_dataset, truncation=True, padding=True, max_length=130_000, return_tensors="pt")

>>>>>>>>>>> Tokenizing dataset... <<<<<<<<<<<


100%|██████████| 1087/1087 [00:03<00:00, 305.53it/s]


In [17]:
train_dataset_encodings['input_ids'][1]

tensor([[128000,    198,    257,  ...,    851,    397,    257]])

In [18]:
train_dataset = CustomDatasetWebCrawler(train_dataset_encodings)
val_dataset = CustomDatasetWebCrawler(val_dataset_encodings)

In [19]:
sampler1 = DistributedSampler(train_dataset, num_replicas=2, rank=0)
sampler2 = DistributedSampler(val_dataset, num_replicas=2, rank=0)

In [20]:
type(sampler1), type(sampler2)

(torch.utils.data.distributed.DistributedSampler,
 torch.utils.data.distributed.DistributedSampler)

In [21]:
train_kwargs = {'batch_size': 2, 'sampler': sampler1}
val_kwargs = {'batch_size': 2, 'sampler': sampler2}
cuda_kwargs = {'num_workers': 8, 'pin_memory': True, 'shuffle': False} # Specify `num_workers=8`, Intel® Core™ i7-11700 Desktop Processor 8 Cores up to 4.9 GHz LGA1200 (Intel® 500 Series & Select 400 Series Chipset) 65W
train_kwargs.update(cuda_kwargs)
val_kwargs.update(cuda_kwargs)

In [22]:
train_kwargs, val_kwargs

({'batch_size': 2,
  'sampler': <torch.utils.data.distributed.DistributedSampler at 0x7a085e5c1670>,
  'num_workers': 8,
  'pin_memory': True,
  'shuffle': False},
 {'batch_size': 2,
  'sampler': <torch.utils.data.distributed.DistributedSampler at 0x7a085e5c1370>,
  'num_workers': 8,
  'pin_memory': True,
  'shuffle': False})

In [23]:
train_loader = DataLoader(train_dataset, **train_kwargs)
val_loader = DataLoader(val_dataset, **val_kwargs)

In [24]:
for i in train_loader:
    print(i)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/rhome/f111169109/.conda/envs/torchtune/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/rhome/f111169109/.conda/envs/torchtune/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/rhome/f111169109/.conda/envs/torchtune/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 317, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/rhome/f111169109/.conda/envs/torchtune/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 155, in collate
    clone.update({key: collate([d[key] for d in batch], collate_fn_map=collate_fn_map) for key in elem})
  File "/rhome/f111169109/.conda/envs/torchtune/lib/python3.8/site-packages/